# Converting Homestuck Collection data to an agnostic dataset

This notebook has all the steps for taking data from the [Unofficial Homestuck Collection](https://bambosh.dev/unofficial-homestuck-collection/)'s asset pack and converting it into a dataset that can be used for training a machine learning model.

Some planned uses for this are:
- Summarization
- Style transfer/LORAs
- Chatbots

# Constants and Imports

In [505]:
"""
IMPORTS
Put all at the beginning because I hate notebooks so much
"""
import os
import pandas as pd
import numpy as np
import json
import re
import random
import base64

from openai import OpenAI

from dotenv import load_dotenv

In [506]:

"""
CONSTANTS
This will extract constants from the env variables set in the .env file 
and make them accessible to the notebook
"""
print("Loading variables from .env file\n...")
load_dotenv()

ASSET_PACK_FOLDER = os.getenv("ASSET_PACK_FOLDER")
OUTPUT_FOLDER = os.getenv("OUTPUT_FOLDER")
OPENAI_API_KEY = os.getenv("OPENAI_API")
MODEL_ID = os.getenv("MODEL_ID")
print("Loaded variables successfully")
    
print("Loading constants\n...")
# Relevant folders and files

# Bespoke input files that aren't available elsewhere
INPUT_FOLDER = os.path.join(os.path.dirname(os.path.abspath('')), "input")

# Transcripts and commentary from ReadMSPA, assembled by Bambosh, Makin and Giovanh
MSPA_COMMENTARY = os.path.join(INPUT_FOLDER, "commentary.json")

# Full transcripts from ReadMSPA, assembled by Giovanh and Bambosh
READMSPA_ALTNARRATIVE = os.path.join(INPUT_FOLDER, "altnarrative.json")
READMSPA_ALTTEXT = os.path.join(INPUT_FOLDER, "alttext.json")
READMSPA_TRANSCRIPTS = os.path.join(INPUT_FOLDER, "transcripts.json")

# Panel tags from the Homestuck Search Engine
HSSE_TAGS = os.path.join(INPUT_FOLDER, "hsse_tags.json")
HSSE_SEARCH = os.path.join(INPUT_FOLDER, "hsse_search.json")

# POV cam data folder with txt files
POV_CAM_FOLDER = os.path.join(INPUT_FOLDER, "readable_timelines")

# Homestuck Collection's asset pack data folder
COLLECTION_DATA_FOLDER = os.path.join(ASSET_PACK_FOLDER, "archive/data")

# Holds all the text in MS Paint Adventures, including Homestuck
MSPA_TEXT_JSON = os.path.join(COLLECTION_DATA_FOLDER, "mspa.json")
# Holds the text for news posts
NEWS_JSON = os.path.join(COLLECTION_DATA_FOLDER, "news.json")
# Holds the text for social media posts
SOCIAL_JSON = os.path.join(COLLECTION_DATA_FOLDER, "social.json")
# Most of this is irrelevant, but holds images for additional Hussie comics
# such as Team Special Olympics
ADDITIONAL_COMICS_JSON = os.path.join(COLLECTION_DATA_FOLDER, "comics.json")
# Holds panels
PANELS_FOLDER = os.path.join(ASSET_PACK_FOLDER, "storyfiles")
HS_PANELS_FOLDER = os.path.join(PANELS_FOLDER, "hs2")

print("Loaded constants successfully")
      
openai_client = OpenAI(api_key=OPENAI_API_KEY)

print(f"OpenAI client loaded with model {MODEL_ID}")

Loading variables from .env file
...
Loaded variables successfully
Loading constants
...
Loaded constants successfully
OpenAI client loaded with model gpt-4o


# Extract MSPA data from Asset Pack

First of all, we want to open the MSPA_TEXT_JSON and fetch all the text data from it. This will be the main source of text data for our dataset.

In [507]:
"""
MSPA_TEXT_JSON format:
{
  "story": {  # The text in MS Paint Adventures, the comics themselves
    "000006": {
      "title": "Look for keyhole",
      "pageId": "000006",
      "timestamp": "1180921880",
      "flag": [],
      "media": [
        "/advimgs/jb/mspaintadventure04.gif",
        "/advimgs/jb/mspaintadventure04b.gif"
      ],
      "content": "",
      "next": [
        "000008"
      ],
      "previous": "000005",
      "theme": "retro"
    },
    "000009": {
      "title": "Loudly tell that guy to pick up key and try it on the door.",
      "pageId": "000009",
      "timestamp": "1180931172",
      "flag": [],
      "media": [
        "/advimgs/jb/mspaintadventure06.gif"
      ],
      "content": "Despite your bellowing, the man casually opens the door and leaves.",
      "next": [
        "000010"
      ],
      "previous": "000008",
      "theme": "retro"
    },
  },...
  "ryanquest": {...},  # Additional Ryanquest comic
  
  "faqs": {
    "general": {
      "title": "General FAQ - MS Paint Adventures",
      "pageId": "general",
      "content": "..." # html
    },
    "new": {...},  # New reader guide
    "science": {...},  # Science FAQ
    "sales": {...}, # This one was probably not Hussie, so ignore
  },
  "psExtras": {...}, # Bonus pages for Problem Sleuth with multiple images, same format with "title", "pageId" and "content" as the subkeys in "faqs"
  "wv": {...}, # "Exile" Homestuck pages with multiple images, same format as psExtras
  # Other keys are fully irrelevant
"""

def get_regular_story_records(mspa_data: dict, section_name: str):
    records = []
    section_data = mspa_data[section_name]
    for page_id, page_content in section_data.items():
        # Extract relevant fields
        record = {
            'section': section_name,
            'pageId': page_id,
            'title': page_content.get('title', ''),
            'content': page_content.get('content', ''),
            'timestamp': page_content.get('timestamp', ''),
            'media': page_content.get('media', []),
            'next': page_content.get('next', []),
            'previous': page_content.get('previous', ''),
            'theme': page_content.get('theme', ''),
            'flag': page_content.get('flag', []),
        }
        records.append(record)
    return records

def get_media_from_html_content(content: str):
    # Extract media from HTML content
    media = []
    # Extract media URLs from the content
    matches = re.findall(r'src="([^"]+)"', content)
    for match in matches:
        # Remove the string "http://www.mspaintadventures.com" and "https://www.mspaintadventures.com" from the beginning if it exists
        match_cleaned = re.sub(r'^https?://www\.mspaintadventures\.com', '', match)
        media.append(match_cleaned)
    return media

def get_onepage_story_records(mspa_data: dict, section_name: str):
    # We need this one for faqs/psExtras/wv, which keep all information in one page and need the panels manually parsed
    records = []
    section_data = mspa_data[section_name]
    for page_id, page_content in section_data.items():
        # Extract relevant fields
        record = {
            'section': section_name,
            'pageId': page_id,
            'title': page_content.get('title', ''),
            'content': page_content.get('content', ''),
            'timestamp': None,
            'media': get_media_from_html_content(page_content.get('content', '')),
            'next': None,
            'previous': None,
            'theme': None,
            'flag': None,
        }
        records.append(record)
    return records

# Function to load and process MSPA text data from mspa.json
def load_mspa_text_data(mspa_json_path):
    """
    Loads and processes MSPA text data from the given JSON file.

    Args:
        mspa_json_path (str): Path to the mspa.json file.

    Returns:
        pd.DataFrame: A DataFrame containing the MSPA text data.
    """
    import json
    import pandas as pd

    # Load the JSON data
    with open(mspa_json_path, 'r', encoding='utf-8') as f:
        mspa_data = json.load(f)

    # Initialize an empty list to store records
    records = []
    
    # Print unique sections
    print(f"Unique sections in mspa.json: {list(mspa_data.keys())}")

    # Sections to process (basically we just skip FAQs)
    records += get_regular_story_records(mspa_data, 'story')
    records += get_regular_story_records(mspa_data, 'ryanquest')
    records += get_onepage_story_records(mspa_data, 'wv')
    records += get_onepage_story_records(mspa_data, 'psExtras')

    # Create a DataFrame from the records
    df = pd.DataFrame(records)

    return df

# Now call the function and store the result in a DataFrame
mspa_df = load_mspa_text_data(MSPA_TEXT_JSON)
mspa_df

Unique sections in mspa.json: ['story', 'ryanquest', 'psExtras', 'wv', 'faqs', 'damara', 'oilRetcon']


,section,pageId,title,content,timestamp,media,next,previous,theme,flag
0,story,000006,Look for keyhole,,1180921880,"[/advimgs/jb/mspaintadventure04.gif, /advimgs/...",[000008],000005,retro,[]
1,story,000009,Loudly tell that guy to pick up key and try it...,"Despite your bellowing, the man casually opens...",1180931172,[/advimgs/jb/mspaintadventure06.gif],[000010],000008,retro,[]
2,story,000010,Pee out the window in an attempt to bring him ...,"As you turn around to unzip, you notice anothe...",1180931381,"[/advimgs/jb/mspaintadventure07.gif, /advimgs/...",[000011],000009,retro,[]
3,story,000011,Open door,Wonders where the fuck that pumpkin went???,1180931414,[/advimgs/jb/mspaintadventure08.gif],[000012],000010,retro,[]
4,story,000005,Grow two muscular arms with which the manhandl...,"He already has arms, stupid!",1180921870,[/advimgs/jb/mspaintadventure03.gif],[000006],000004,retro,[]
...,...,...,...,...,...,...,...,...,...,...
10049,psExtras,ps000037,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,"[/extras/ps000037_1.gif, /extras/ps000037_2.gi...",None,None,None,None
10050,psExtras,ps000039,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,"[/extras/ps000039_1.gif, /extras/ps000039_2.gif]",None,None,None,None
10051,psExtras,ps000034,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,"[/extras/ps000034_1.gif, /extras/ps000034_2.gi...",None,None,None,None
10052,psExtras,ps000031,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,"[/extras/ps000031_1.gif, /extras/ps000031_2.gi...",None,None,None,None


In [508]:
# Show pageId == anunsealedrecord, a special case
mspa_df[mspa_df['pageId'] == 'anunsealedtunnel']

,section,pageId,title,content,timestamp,media,next,previous,theme,flag
10006,wv,anunsealedtunnel,MS Paint Adventures,"\n\n <tbody><tr>\n <td bgcolor=""#FFFFFF""...",None,[/storyfiles/hs2/waywardvagabond/anunsealedtun...,None,None,None,None


# Creating vizPageId from pageId

The Homestuck Collection uses the MSPA page IDs, but the Homestuck website uses its own page IDs. We can convert the MSPA page IDs to the Homestuck website page IDs with some clever and totally not bruteforced heuristics. We will need to do this sooner or later because multiple upcoming datasets use the Viz page ids, which are missing in mspa_df right now.

In [509]:
# Define the adventure number to story name mapping
adventure_mapping = {
    0: "Other Section",
    1: "Jailbreak",
    2: "Bard Quest",
    3: "Blood Spade",
    4: "Problem Sleuth",
    5: "Homestuck BETA",
    6: "Homestuck",
    "ryanquest": "Ryanquest",
    "psExtras": "Problem Sleuth Extras"
}

# Define the story name to URL path mapping
story_url_mapping = {
    "Jailbreak": "jailbreak",
    "Bard Quest": "bard-quest",
    "Blood Spade": "blood-spade",
    "Problem Sleuth": "problem-sleuth",
    "Homestuck BETA": "beta",
    "Homestuck": "story",
    "Ryanquest": "ryanquest",
    "Other Section": "story/waywardvagabond"
}

def get_story_num(page_id):
    """
    Determines the story ID based on the page number.
    
    Args:
        page_id (str): The pageId, can be numeric or special identifier.
    
    Returns:
        int or str: The story number (1-6) or 'Unknown'
    """
    if page_id.isdigit():
        page_num = int(page_id.lstrip('0'))
        if page_num <= 135:
            return 1  # Jailbreak
        elif 136 <= page_num <= 216:
            return 2  # Bard Quest
        elif 219 <= page_num <= 1892:
            return 4  # Problem Sleuth
        elif 1893 <= page_num <= 1900:
            return 5  # Homestuck BETA
        elif 1901 <= page_num <= 10030:
            return 6  # Homestuck
        else:
            return "Unknown"
    else:
        # Handle special cases
        if page_id == 'mc0001':
            return 3  # Blood Spade
        elif page_id in ['pony', 'pony2', 'darkcage', 'darkcage2']:
            return 6  # Homestuck
        else:
            return "Unknown"

def mspa_to_viz(row):
    """
    Converts a row's pageId and section to vizStory, vizPageNumber, vizPageId.
    
    Args:
        row (pd.Series): A row from the DataFrame.
    
    Returns:
        pd.Series: A series with 'vizStory', 'vizPageNumber', and 'vizPageId'.
    """
    section = row['section'].lower()
    page_id = row['pageId']
    
    if section == 'ryanquest':
        viz_story = "Ryanquest"
        try:
            viz_page_number = int(page_id.lstrip('0'))
        except:
            viz_page_number = page_id  # Preserve special strings if any
        viz_page_id = f"ryanquest/{viz_page_number}" if isinstance(viz_page_number, int) else f"ryanquest/{viz_page_number}"
    elif section == 'story':
        story_num = get_story_num(page_id)
        if isinstance(story_num, int) and story_num in adventure_mapping:
            viz_story = adventure_mapping[story_num]
            # Adjust vizPageNumber based on story_num
            try:
                if story_num == 1:  # Jailbreak
                    viz_page_number = int(page_id.lstrip('0'))
                elif story_num == 2:  # Bard Quest
                    viz_page_number = int(page_id.lstrip('0')) - 135
                elif story_num == 3:  # Blood Spade
                    viz_page_number = "1"  # Only 'mc0001'
                elif story_num == 4:  # Problem Sleuth
                    viz_page_number = int(page_id.lstrip('0')) - 218
                elif story_num == 5:  # Homestuck BETA
                    viz_page_number = int(page_id.lstrip('0')) - 1892
                elif story_num == 6:  # Homestuck
                    if page_id in ['pony', 'pony2', 'darkcage', 'darkcage2']:
                        viz_page_number = page_id  # Preserve special strings
                    else:
                        viz_page_number = int(page_id.lstrip('0')) - 1900
                else:
                    viz_page_number = None
            except:
                viz_page_number = None
            
            # Construct vizPageId based on vizPageNumber
            if isinstance(viz_page_number, int):
                viz_page_id = f"{story_num}/{viz_page_number}"
            elif isinstance(viz_page_number, str):
                viz_page_id = f"{story_num}/{viz_page_number}"
            else:
                viz_page_id = f"{story_num}/000000"
        else:
            viz_story = "Unknown Adventure"
            viz_page_number = None
            viz_page_id = "Unknown/000000"
    else:
        viz_story = "Other Section"
        try:
            viz_page_number = int(page_id.lstrip('0'))
        except:
            viz_page_number = page_id  # Preserve special strings if any
        viz_page_id = f"other/{viz_page_number}" if isinstance(viz_page_number, int) else f"other/{viz_page_number}"
    
    return pd.Series({
        'vizStory': viz_story,
        'vizPageNumber': viz_page_number,
        'vizPageId': viz_page_id
    })

def create_viz_link(row):
    """
    Creates a visualization link based on vizStory and vizPageNumber.
    
    Args:
        row (pd.Series): A row from the DataFrame.
    
    Returns:
        str or None: The constructed URL or None if information is missing.
    """
    viz_story = row['vizStory']
    viz_page_number = row['vizPageNumber']
    
    if pd.isnull(viz_story) or pd.isnull(viz_page_number):
        return None
    
    base_url = "https://homestuck.com/"
    story_path = story_url_mapping.get(viz_story, "other")
    
    # Handle potential edge cases where viz_page_number is not an integer
    if isinstance(viz_page_number, float):
        if viz_page_number.is_integer():
            viz_page_number = int(viz_page_number)
        else:
            # If it's not an integer, handle accordingly (e.g., set to None or round)
            return None
    
    return f"{base_url}{story_path}/{viz_page_number}"

mspa_df['pageId'] = mspa_df['pageId'].astype(str)

# Apply the mspa_to_viz function to create 'vizStory', 'vizPageNumber', and 'vizPageId' columns
mspa_df[['vizStory', 'vizPageNumber', 'vizPageId']] = mspa_df.apply(mspa_to_viz, axis=1)

# Create the 'vizLink' column
mspa_df['vizLink'] = mspa_df.apply(create_viz_link, axis=1)

# Convert 'vizPageNumber' to integer where possible, else keep as string for special cases
def convert_viz_page_number(x):
    if isinstance(x, int):
        return x
    elif isinstance(x, str):
        return x  # Preserve special strings
    else:
        return pd.NA

mspa_df['vizPageNumber'] = mspa_df['vizPageNumber'].apply(convert_viz_page_number)

# Display the updated DataFrame sample
mspa_df[['section', 'title', 'pageId', 'content', 'vizStory', 'vizPageNumber', 'vizPageId', 'vizLink']].sample(10)

,section,title,pageId,content,vizStory,vizPageNumber,vizPageId,vizLink
1073,story,AS: Give all the weapons to PI.,001111,"You order PI to take the KEY, the RING OF KEYS...",Problem Sleuth,893,4/893,https://homestuck.com/problem-sleuth/893
499,story,PI: Fire!,000537,The JAW BREAKER CANNON inflicts a SAVAGE UPBRA...,Problem Sleuth,319,4/319,https://homestuck.com/problem-sleuth/319
4061,story,Vriska: Take dice.,004099,"You equip your enchanted dice set, the fabled ...",Homestuck,2199,6/2199,https://homestuck.com/story/2199
7041,story,==>,007082,"|PESTERLOG|<br /><span style=""color: #323232"">...",Homestuck,5182,6/5182,https://homestuck.com/story/5182
6055,story,Jane: Examine portrait.,006096,Just one of your dad's bland HALLWAY DOUCHEBAG...,Homestuck,4196,6/4196,https://homestuck.com/story/4196
4171,story,======>,004208,She could not stand bearing the visage of the ...,Homestuck,2308,6/2308,https://homestuck.com/story/2308
782,story,AD: Go downstairs and check out swimming pool.,000823,It is a luxurious above ground pool. It looks ...,Problem Sleuth,605,4/605,https://homestuck.com/problem-sleuth/605
5786,story,[o],005828,,Homestuck,3928,6/3928,https://homestuck.com/story/3928
5247,story,==>,005288,"Speak to me, Rufio. <br /><br />Oh god what ha...",Homestuck,3388,6/3388,https://homestuck.com/story/3388
9068,story,[A6A6I4] ====>,009107,"|DIALOGLOG|<br /><span style=""color: #0715cd"">...",Homestuck,7207,6/7207,https://homestuck.com/story/7207


# Unpacking the page-based dataset into a panel-based dataset

The previous works fine for data sourced on pages instead of panels, like commentaries, but we're merging with ReadMSPA next, so we will need to `explode` the data and do some more edits, since the panel srcs will need some editing to match the transcripts.

In [510]:
mspa_panels_df = mspa_df.explode('media').reset_index(drop=True)
# Rename 'media' to 'src' for clarity
mspa_panels_df.rename(columns={'media': 'src'}, inplace=True)
# Get a sample of Homestuck panels
mspa_panels_df[mspa_panels_df['vizStory'] == 'Homestuck'].head(10)

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,vizStory,vizPageNumber,vizPageId,vizLink
2398,story,001902,Enter name.,,1239607342,/storyfiles/hs2/00002.gif,[001903],001901,,[],Homestuck,2,6/2,https://homestuck.com/story/2
2399,story,001901,Homestuck,A young man stands in his bedroom. It just so ...,1239607316,/storyfiles/hs2/00001.gif,[001902],,,[],Homestuck,1,6/1,https://homestuck.com/story/1
2400,story,001904,Examine room.,Your name is JOHN. As was previously mentioned...,1239607407,/storyfiles/hs2/00004.gif,[001905],001903,,[],Homestuck,4,6/4,https://homestuck.com/story/4
2401,story,001903,Try again.,,1239607364,/storyfiles/hs2/00003.gif,[001904],001902,,[],Homestuck,3,6/3,https://homestuck.com/story/3
2402,story,001908,John: Examine contents of chest.,In here you keep an array of humorous and myst...,1239678906,/storyfiles/hs2/00008.gif,[001909],001907,,[],Homestuck,8,6/8,https://homestuck.com/story/8
2403,story,001905,John: Quickly retrieve arms from drawer.,"Your ARMS are in your MAGIC CHEST, pooplord!",1239607433,/storyfiles/hs2/00005.gif,[001906],001904,,[],Homestuck,5,6/5,https://homestuck.com/story/5
2404,story,001909,John: Captchalogue smoke pellets.,You stow the SMOKE PELLETS on one of your CAPT...,1239678933,/storyfiles/hs2/00009.gif,[001910],001908,,[],Homestuck,9,6/9,https://homestuck.com/story/9
2405,story,001907,John: Quickly retrieve arms from MAGIC CHEST.,You retrieve your FAKE ARMS from the chest. Yo...,1239607518,/storyfiles/hs2/00007_1.gif,[001908],001906,,[],Homestuck,7,6/7,https://homestuck.com/story/7
2406,story,001907,John: Quickly retrieve arms from MAGIC CHEST.,You retrieve your FAKE ARMS from the chest. Yo...,1239607518,/storyfiles/hs2/00007_2.gif,[001908],001906,,[],Homestuck,7,6/7,https://homestuck.com/story/7
2407,story,001906,Remove CAKE from MAGIC CHEST.,Out of sympathy for John's perceived lack of a...,1239607471,/storyfiles/hs2/00006.gif,[001907],001905,,[],Homestuck,6,6/6,https://homestuck.com/story/6


In [511]:
# Some panel transcripts will have an id format like `00001.gif.transcript`, which in our current src is `/storyfiles/hs2/00001.gif`. We need to make a transcriptSrc to match the panel src. This applies no matter what the comics is, so we can't assume it will be /storyfiles/hs2/... we just pick the actual image file name and add '.transcript' to it.
def create_transcript_src(src):
    if pd.isnull(src):
        return None
    
    # Extract the image file name from the src
    image_file = src.split('/')[-1]
    # Append '.transcript' to the image file name
    transcript_src = f"{image_file}.transcript"
    
    return transcript_src
    
# Apply the function to create 'transcriptSrc' column
mspa_panels_df['transcriptSrc'] = mspa_panels_df['src'].apply(create_transcript_src)
mspa_panels_df[mspa_panels_df['vizStory'] == 'Homestuck'].head(10)

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,vizStory,vizPageNumber,vizPageId,vizLink,transcriptSrc
2398,story,001902,Enter name.,,1239607342,/storyfiles/hs2/00002.gif,[001903],001901,,[],Homestuck,2,6/2,https://homestuck.com/story/2,00002.gif.transcript
2399,story,001901,Homestuck,A young man stands in his bedroom. It just so ...,1239607316,/storyfiles/hs2/00001.gif,[001902],,,[],Homestuck,1,6/1,https://homestuck.com/story/1,00001.gif.transcript
2400,story,001904,Examine room.,Your name is JOHN. As was previously mentioned...,1239607407,/storyfiles/hs2/00004.gif,[001905],001903,,[],Homestuck,4,6/4,https://homestuck.com/story/4,00004.gif.transcript
2401,story,001903,Try again.,,1239607364,/storyfiles/hs2/00003.gif,[001904],001902,,[],Homestuck,3,6/3,https://homestuck.com/story/3,00003.gif.transcript
2402,story,001908,John: Examine contents of chest.,In here you keep an array of humorous and myst...,1239678906,/storyfiles/hs2/00008.gif,[001909],001907,,[],Homestuck,8,6/8,https://homestuck.com/story/8,00008.gif.transcript
2403,story,001905,John: Quickly retrieve arms from drawer.,"Your ARMS are in your MAGIC CHEST, pooplord!",1239607433,/storyfiles/hs2/00005.gif,[001906],001904,,[],Homestuck,5,6/5,https://homestuck.com/story/5,00005.gif.transcript
2404,story,001909,John: Captchalogue smoke pellets.,You stow the SMOKE PELLETS on one of your CAPT...,1239678933,/storyfiles/hs2/00009.gif,[001910],001908,,[],Homestuck,9,6/9,https://homestuck.com/story/9,00009.gif.transcript
2405,story,001907,John: Quickly retrieve arms from MAGIC CHEST.,You retrieve your FAKE ARMS from the chest. Yo...,1239607518,/storyfiles/hs2/00007_1.gif,[001908],001906,,[],Homestuck,7,6/7,https://homestuck.com/story/7,00007_1.gif.transcript
2406,story,001907,John: Quickly retrieve arms from MAGIC CHEST.,You retrieve your FAKE ARMS from the chest. Yo...,1239607518,/storyfiles/hs2/00007_2.gif,[001908],001906,,[],Homestuck,7,6/7,https://homestuck.com/story/7,00007_2.gif.transcript
2407,story,001906,Remove CAKE from MAGIC CHEST.,Out of sympathy for John's perceived lack of a...,1239607471,/storyfiles/hs2/00006.gif,[001907],001905,,[],Homestuck,6,6/6,https://homestuck.com/story/6,00006.gif.transcript


In [512]:
# Now we need to do something similar for MSPA URLs to match the key format for pages of the READMSPA format. We have a pageId like 001901, but we need to get the story_num using the matching we already have, and prepend that to the page number. So 001901 would become '6/001901' because that page number matches Homestuck. This can be the new column "pageNumber" because it's the vizPageNumber in MSPA format..
def create_page_number(row):
    """
    Creates a page number based on vizStory and vizPageNumber.
    
    Args:
        row (pd.Series): A row from the DataFrame.
    
    Returns:
        str or None: The constructed page number or None if information is missing.
    """
    
    viz_page_number = row['pageId']
    
    if pd.isnull(viz_page_number):
        return None
        
    viz_story_number = get_story_num(str(viz_page_number))
    
    return f"{viz_story_number}/{viz_page_number}"

# Apply the function to create 'pageNumber' column
mspa_panels_df['pageNumber'] = mspa_panels_df.apply(create_page_number, axis=1)

# Display the updated DataFrame sample
mspa_panels_df[mspa_panels_df['vizStory'] == 'Homestuck'].head(10)

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,vizStory,vizPageNumber,vizPageId,vizLink,transcriptSrc,pageNumber
2398,story,001902,Enter name.,,1239607342,/storyfiles/hs2/00002.gif,[001903],001901,,[],Homestuck,2,6/2,https://homestuck.com/story/2,00002.gif.transcript,6/001902
2399,story,001901,Homestuck,A young man stands in his bedroom. It just so ...,1239607316,/storyfiles/hs2/00001.gif,[001902],,,[],Homestuck,1,6/1,https://homestuck.com/story/1,00001.gif.transcript,6/001901
2400,story,001904,Examine room.,Your name is JOHN. As was previously mentioned...,1239607407,/storyfiles/hs2/00004.gif,[001905],001903,,[],Homestuck,4,6/4,https://homestuck.com/story/4,00004.gif.transcript,6/001904
2401,story,001903,Try again.,,1239607364,/storyfiles/hs2/00003.gif,[001904],001902,,[],Homestuck,3,6/3,https://homestuck.com/story/3,00003.gif.transcript,6/001903
2402,story,001908,John: Examine contents of chest.,In here you keep an array of humorous and myst...,1239678906,/storyfiles/hs2/00008.gif,[001909],001907,,[],Homestuck,8,6/8,https://homestuck.com/story/8,00008.gif.transcript,6/001908
2403,story,001905,John: Quickly retrieve arms from drawer.,"Your ARMS are in your MAGIC CHEST, pooplord!",1239607433,/storyfiles/hs2/00005.gif,[001906],001904,,[],Homestuck,5,6/5,https://homestuck.com/story/5,00005.gif.transcript,6/001905
2404,story,001909,John: Captchalogue smoke pellets.,You stow the SMOKE PELLETS on one of your CAPT...,1239678933,/storyfiles/hs2/00009.gif,[001910],001908,,[],Homestuck,9,6/9,https://homestuck.com/story/9,00009.gif.transcript,6/001909
2405,story,001907,John: Quickly retrieve arms from MAGIC CHEST.,You retrieve your FAKE ARMS from the chest. Yo...,1239607518,/storyfiles/hs2/00007_1.gif,[001908],001906,,[],Homestuck,7,6/7,https://homestuck.com/story/7,00007_1.gif.transcript,6/001907
2406,story,001907,John: Quickly retrieve arms from MAGIC CHEST.,You retrieve your FAKE ARMS from the chest. Yo...,1239607518,/storyfiles/hs2/00007_2.gif,[001908],001906,,[],Homestuck,7,6/7,https://homestuck.com/story/7,00007_2.gif.transcript,6/001907
2407,story,001906,Remove CAKE from MAGIC CHEST.,Out of sympathy for John's perceived lack of a...,1239607471,/storyfiles/hs2/00006.gif,[001907],001905,,[],Homestuck,6,6/6,https://homestuck.com/story/6,00006.gif.transcript,6/001906


# Getting panel transcripts from ReadMSPA plugin

ReadMSPA's data (and its plugin from the collection) comes with text transcripts of every image's text, if not descriptions. We can use that.

In [513]:
"""
The ReadMSPA data is divided in three similar files for unknown reasons, READMSPA_TRANSCRIPTS, READMSPA_ALTTEXT, READMSPA_ALTNARRATIVE. The latter tends to have way longer transcriptions, but there are long transcriptions in READMSPA_ALTTEXT, and they have different formats besides. READMSPA_TRANSCRIPTS even contains transcripts of youtube links linked in the pages, which we should preserve anyway. 
Note the keys are not equivalent to pageId. These keys use the mspaintadventures.com format, where page numbers were padded with zeroes, as well as sharing a count for every adventure in the site, from Jailbreak to Homestuck. Homestuck's first page is 001901 and it's the adventure number 6 (instead of 4 for historical reasons, but this is not really relevant to us right now).

Portion of READMSPA_TRANSCRIPTS (the key is a pageNumber, and the array is further discriminated by the id property, which will match transcriptSrc):
```json
{
    "6/001901": [
        {
            "src": null,
            "alt": "SBURB BETA",
            "contents": "SBURB BETA",
            "id": "00001.gif.transcript",
            "data": null,
            "class": [
                "transcript"
            ],
            "style": "background: white; color: #C1C1C1",
            "tag": "div"
        }
    ],
    "6/010030": [
        {
            "src": "http/www.youtube.com-rmzu89jy2j8.mp4",
            "alt": "* (This transcribes both the snapchat pics from the right of the video - which as narrative, are included in readmspa.org word count stats - and the credits from the left, which are not.)\n\n\nmspaofficial\n\nAdded Me\nAdd Friends\nMy Friends\n\n\n* HOMESTUCK\n\n* BY\n* ANDREW HUSSIE\n\n\nGreetings From\nEARTH C\n\n\n* SOUND\n* CONTRIBUTORS (A-Z)\n\n* Alex Rosetti\n* Andrew Huo\n* BurnedKirby\n* Charlie Clouser\n* Clark Powell\n* Curt Blakeslee\n* David DeCou\n* David Ko\n\n\nMAYOR\n\n\n* Dianne Warren\n* DJ Sai Tae\n* Eddie Morton\n* Eligecos\n* Erik Scheele\n* Eston Schweickart\n* First Turn Fold\n* Frank Haught\n* Gabe Nezovic\n\n\n* Insane Clown Posse\n* James Roach\n* Jan Van Den Hemel\n* Joseph Aylsworth\n* Kalibration\n* Kera Jones\n* Kevin Regamey\n* Kezinox\n* Malcolm Brown\n\n\n* Mark Hadley\n* Michael Guy Bowman\n* Michael Vallejo\n* Nick Smalley\n* Noel Sadwin\n* Paul Tuttle Starr\n* Perry Sullivan\n* Plumegeist\n* Robert J! Lake\n\n\n* Samm 413\ntentacleTherapist\ngallowsCallibrator\ntipsyGnostalgic\ngrimAuxiliatrix\ngolgothasTerror\ntimaeusTestified\ngutsyGumshoe\n\n\nCALIBORN: COME AT ME BRO.\n\n\nJOHN: step off.\n\n\nCALIBORN: COME TO MY DARK CARNIVAL. \"MOTHER FUCKER\".\n\n\nJOHN: i'll do it...\n\n\nCALIBORN: MAKE A MOVE. AND THE BUNNY GETS IT.",
            "contents": "",
            "id": "08123.mp4",
            "data": "End Credits",
            "class": [
                "flash",
                "alt-narrative"
            ],
            "style": null,
            "tag": "embed"
        }
    ],
```

Portion of READMSPA_ALTTEXT (a key matches the src column):
```json
{
    "advimgs/jb/mspaintadventure06.gif": "\nBELLOW!\n\nSLAM\n",
    "advimgs/jb/mspaintadventure09d.gif": "CRACK",
    "advimgs/jb/mspaintadventure09e.gif": "SLAM",
    "advimgs/jb/mspaintadventure05.gif": "CLANK",
    "advimgs/jb/mspaintadventure08.gif": "? ? ? ?",
    ...
    "storyfiles/hs2/07655.gif": "\nDAVE: ...\nDIRK: ...\n",
    "storyfiles/hs2/07651.gif": "nuzzzzzzzzzle",
    "storyfiles/hs2/07650.gif": "nuzzzzle",
    "storyfiles/hs2/07648.gif": "KANAYA: Shoutpole",
}

Portion of READMSPA_ALTNARRATIVE (the key is a pageNumber, and the array is further discriminated by the id property, which will match transcriptSrc):
```json
"6/002153": [
        {
            "src": null,
            "alt": "<style>.page[id=\"6/002153\"] .walkaround p { border-color: #23CE27; border-radius: 0;\n\t\t       \t    \t\t    background-color: white; max-width: 100%; }</style>\n* Derived in part from <a href=\"http://pastebin.com/a3k6RgMR\">a transcript by ShadowOfFate</a>\n\n<div style=\"padding: 1em; background-color: white; border: solid 3px black\">CLICK THIS</div><dl>\n<dt>* click it!</dt>\n<dd>\n<p>To walk around, use the mouse, arrow keys, or WASD keys. Click on various objects to open command menus for them!\n\nOutstanding Flash programming by Alexis 'Gankro' Beingessner.</p>\n</dd>\n\n<dt>* click jestersprite</dt>\n<dd>\n<kbd>&gt; WHAT'S THAT\n</kbd>\n<p>It looks different now.\n\nAfter you bit that APPLE, your whole house seemed to be trasported somewhere. Then the APPLE disappeared and the KERNELSPRITE underwent a transformation. Aside from the change in appearance, the transformation doesn't seem to have any relevant ramifications. You still can't understand a word this idiot says.</p>\n<kbd>&gt; THE GHOST CLOWN...",
            "id": "00253.swf.transcript",
            "data": "John explores his house with WV (transcript and walkthrough)"
        }
    ],
    "6/002149": [
        {
            "src": null,
            "alt": "* Derived from <a href=\"http://mspaintadventures.wikia.com/wiki/Rose%27s_Walkthrough\">MSPA Wiki</a>\n\n<p style=\"background-color: black; color: white; padding: 1em;\n          font-family: Lucida Console, courier;\"><b>Sburb Beta FAQ/Walkthrough</b> by <span style=\"color: #ff6600\">tentacleTherapist</span>\n<b>Hosted by</b> <span style=\"color: #ff6600\">GameFAQs</span>\nVersion 1.0, Last Updated 2009-04-13\n<span style=\"color: #ff6600\">View/Download Original File</span>\nLiked this FAQ? Click to <span style=\"color: #ff6600\">recommend</span> it to other GameFAQs users.\n</p>\n<article style=\"font-family: Lucids Console, courier\">\nSburb Beta Walkthrough\nVersion 1.0, April 13, 2009\nBy tentacleTherapist\n\n=============================== TABLE OF CONTENTS ===============================\n\n1. Caveats and Condolences........................... [0000]\n2. Walkthrough (Incomplete).......................... [A000]\n2.1. An Examination of the Basics.................... [A000]\n2.2. So your cruxtruder is ticking. Do this to live.. [A100]\n2.3. The Long and Short. The Medium too.............. [B100]\nsome stuff about captcha codes and punch card alchemy [Z001]\nC. Appendix 3 -- Screen Captures, pt. 1.............. [Z301]\n?. Rose: Egress...",
            "id": "00249_2.gif.transcript",
            "data": "Rose's Sburb Beta Walkthrough - Caveats and Condolences"
        }
    ],
    "6/002037": [
        {
            "src": null,
            "alt": "* Derived from <a href=\"http://pastebin.com/FQhpyeKx\">a transcript by ShadowOfFate</a>\n\n<ul style=\"list-style: none; background-color: #4CE24E; padding: 1em;\n    color: white; text-align: center; white-space: normal\">\n<li>Transforming Soffits</li>\n<li>Reorganizing Keys</li>\n<li>Formalizing Immersion Joints</li>\n<li>Justifying Kick Extractors</li>\n<li>Advising Aggregates</li>\n<li>Managing Elbows</li>...",
            "id": "00137.swf.transcript",
            "data": "Sburb Beta loading screen"
        }
    ]
}

Our objective here is to combine the information so that, for each panel, we'll have its text transcript.
"""

# Extract the ReadMSPA data from the JSON files
def load_readmspa_data(transcripts_path, alttext_path, altnarrative_path):
    """
    Loads and processes the ReadMSPA data from the given JSON files.

    Args:
        transcripts_path (str): Path to the transcripts JSON file.
        alttext_path (str): Path to the alttext JSON file.
        altnarrative_path (str): Path to the altnarrative JSON file.

    Returns:
        pd.DataFrame: A DataFrame containing the ReadMSPA data.
    """
    # Load the JSON data
    with open(transcripts_path, 'r', encoding='utf-8') as f:
        transcripts_data = json.load(f)
    with open(alttext_path, 'r', encoding='utf-8') as f:
        alttext_data = json.load(f)
    with open(altnarrative_path, 'r', encoding='utf-8') as f:
        altnarrative_data = json.load(f)

    # Initialize an empty list to store page records
    page_records = []
    
    # Iterate over each data dictionary of READMSPA_TRANSCRIPTS and READMSPA_ALTNARRATIVE
    for data_dict in [transcripts_data, altnarrative_data]:
        # Iterate over each key-value pair
        for key, value_list in data_dict.items():
            # Extract relevant fields
            for value in value_list:
                record = {
                    'pageNumber': key,
                    'src': value.get('src', None),
                    'contents': value.get('contents', ''),
                    'transcriptSrc': value.get('id', ''),
                    'data': value.get('data', ''),
                    'class': value.get('class', []),
                    'style': value.get('style', ''),
                    'tag': value.get('tag', ''),
                    'alt': value.get('alt', ''),
                }
                page_records.append(record)
            
    # Initialize an empty list to store alttext records
    panel_records = []
    
    # Iterate over each key-value pair in READMSPA_ALTTEXT
    for key, value in alttext_data.items():
        # Extract relevant fields
        record = {
            'src': key,
            'contents': value,
        }
        panel_records.append(record)
        
    # Create DataFrames from the records
    page_df = pd.DataFrame(page_records)
    panel_df = pd.DataFrame(panel_records)
    
    return page_df, panel_df

# Now call the function and store the results in DataFrames
readmspa_pages_df, readmspa_panels_df = load_readmspa_data(READMSPA_TRANSCRIPTS, READMSPA_ALTTEXT, READMSPA_ALTNARRATIVE)

# Display the updated DataFrame samples
readmspa_pages_df.sample(5)
    

,pageNumber,src,contents,transcriptSrc,data,class,style,tag,alt
111,6/005978,None,"<span style=""font-size: 55px"">GIRL, QUIT ALL T...",LEtext3.gif.transcript,LE text,[transcript],background: #082A00; text-align: center; color...,div,"<span style=""font-size: 55px"">GIRL, QUIT ALL T..."
40,6/008668,None,JAKE: OH MY STARS AND GARTERS,06765.gif.transcript,bigjake,"[transcript, hope]",color: #1f9400,div,JAKE: OH MY STARS AND GARTERS
36,6/009138,None,R3SCU3 TH3\nW1TN3SS B3F0R3\nH3 IS MURDERED!\n(...,07235.gif.transcript,bloody scarf,"[transcript, bloody-scarf]",,div,R3SCU3 TH3\nW1TN3SS B3F0R3\nH3 IS MURDERED!\n(...
198,6/005552,storyfiles/hs2/03649_1.gif,,03649_1.gif,AGAIN with the metric sytstem,[],,,AGAIN with the\nmetric sytstem what IS it even...
34,6/009154,None,WR1T3 M3SS4G3\nON W4LL\n\n[???] DON'T-\n[??????],07251.gif.transcript,bloody scarf,"[transcript, bloody-scarf]",,div,WR1T3 M3SS4G3\nON W4LL\n\n[???] DON'T-\n[??????]


In [514]:
readmspa_panels_df.sample(5)

,src,contents
3162,advimgs/ps/ps517.gif,\n5G-PR-HU-62\n9K-Y8-32-BB\nML-L4-DQ-5N\n
1594,storyfiles/hs2/02965.gif,\n--- ....\n-. --- !!!\n
3469,advimgs/jb/mspaintadventure29.gif,\n♩♩♫ ♩♪♩\n\nWHIRRR\n
539,storyfiles/hs2/05800.gif,\nPARKING CITATION\n\n* (22 x 7 = 154 marks on...
2954,advimgs/ps/ps1091.gif,TOOTSIE ROLL FRANKENSTEIN


In [515]:
# Now we can merge readmspa_pages_df on its id (which will match transcriptSrc) with mspa_panels_df on transcriptSrc
# Note we only care about 
mspa_panels_df = mspa_panels_df.merge(readmspa_pages_df, on='transcriptSrc', how='left', suffixes=('', '_readmspapages'))
# Drop duplicate rows, we just want one record per "src".
mspa_panels_df.drop_duplicates(subset='src', inplace=True)
# Show some panels with transcripts
mspa_panels_df[~mspa_panels_df['contents'].isnull()].head(10)

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,...,transcriptSrc,pageNumber,pageNumber_readmspapages,src_readmspapages,contents,data,class,style,tag,alt
2398,story,001902,Enter name.,,1239607342,/storyfiles/hs2/00002.gif,[001903],001901,,[],...,00002.gif.transcript,6/001902,6/001902,None,"<b style=""color: black"">ZOOSMELL POOPLORD</b>\...",None,[transcript],font-family: courier; background: white,div,"<b style=""color: black"">ZOOSMELL POOPLORD</b>\..."
2399,story,001901,Homestuck,A young man stands in his bedroom. It just so ...,1239607316,/storyfiles/hs2/00001.gif,[001902],,,[],...,00001.gif.transcript,6/001901,6/001901,None,SBURB BETA,None,[transcript],background: white; color: #C1C1C1,div,SBURB BETA
2401,story,001903,Try again.,,1239607364,/storyfiles/hs2/00003.gif,[001904],001902,,[],...,00003.gif.transcript,6/001903,6/001903,None,"<b style=""color: green"">✓</b> <b style=""color:...",None,[transcript],font-family: courier; background: white,div,"<b style=""color: green"">✓</b> <b style=""color:..."
2408,story,001912,John: Read note on drawer.,This note is rich with the aromas of FATHERLY ...,1239679025,/storyfiles/hs2/00012.gif,[001913],001911,,[],...,00012.gif.transcript,6/001912,6/001912,None,HAPPY BIRTHDAY SON.\n\nI AM SO PROUD OF YOU.,None,"[transcript, dad-note]",,div,HAPPY BIRTHDAY SON.\n\nI AM SO PROUD OF YOU.
2562,story,002037,[S] ==>,,1241865178,/storyfiles/hs2/00137/00137.swf,[002038],002036,,"[F, HQ]",...,00137.swf.transcript,6/002037,6/002037,None,"* Derived from <a href=""http://pastebin.com/FQ...",Sburb Beta loading screen,"[transcript, alt-narrative]",,div,"* Derived from <a href=""http://pastebin.com/FQ..."
2694,story,002149,ACT 2 ==>,,1244666782,/storyfiles/hs2/00249_2.gif,[002150],002148,,[],...,00249_2.gif.transcript,6/002149,6/002149,None,"* Derived from <a href=""http://mspaintadventur...",Rose's Sburb Beta Walkthrough - Caveats and Co...,"[transcript, alt-narrative]",,div,"* Derived from <a href=""http://mspaintadventur..."
2704,story,002153,[S] YOU THERE. BOY.,,1244949337,/storyfiles/hs2/00253/00253.swf,[002154],002152,,[F],...,00253.swf.transcript,6/002153,6/002153,None,"<style>.page[id=""6/002153""] .walkaround p { bo...",John explores his house with WV (transcript an...,"[transcript, alt-narrative, walkaround]",,div,"<style>.page[id=""6/002153""] .walkaround p { bo..."
2726,story,002172,==>,,1245374550,/storyfiles/hs2/00272_2.gif,[002173],002171,,[],...,00272_2.gif.transcript,6/002172,6/002172,None,"* Derived from <a href=""http://mspaintadventur...",Rose's Sburb Beta Walkthrough - An Examination...,"[transcript, alt-narrative]",,div,"* Derived from <a href=""http://mspaintadventur..."
2919,story,002340,==>,"She's not finished with this yet! Jeez, cut he...",1249444721,/storyfiles/hs2/00440_2.gif,[002341],002339,,[],...,00440_2.gif.transcript,6/002340,6/002340,None,"* Derived from <a href=""http://mspaintadventur...",Rose's Sburb Beta Walkthrough - The Long and S...,"[transcript, alt-narrative]",,div,"* Derived from <a href=""http://mspaintadventur..."
2998,story,002410,==>,"<a href=""http://www.mspaintadventures.com/stor...",1250822032,/storyfiles/hs2/00510_2.gif,[002411],002409,,[],...,00510_2.gif.transcript,6/002410,6/002410,None,"<article style=""font-family: Lucida Console, c...",Rose's Sburb Beta Walkthrough - Screen Captures,"[transcript, alt-narrative]",,div,"<article style=""font-family: Lucida Console, c..."


In [516]:
# Merge the panel data too, this time on readmspa_panels_df's src, which matches mspa_panels_df's src, but, critically, without the slash at the beginning. 
# Make a copy of the src column in readmspa_panels_df to match the src column in mspa_panels_df
mspa_panels_df['src_copy'] = mspa_panels_df['src'].apply(lambda x: x[1:] if x.startswith('/') else x)
# Merge the DataFrames
mspa_panels_annotated_df = mspa_panels_df.merge(readmspa_panels_df, left_on='src_copy', right_on='src', how='left', suffixes=('', '_readmspanels'))
# Show a panel with annotations
mspa_panels_annotated_df[mspa_panels_annotated_df['src'] == '/storyfiles/hs2/01085_2.gif']

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,...,src_readmspapages,contents,data,class,style,tag,alt,src_copy,src_readmspanels,contents_readmspanels
3653,story,002985,Dave: Eject your modus and set it to Scrabble ...,You dump all this crap all over the roof.<br /...,1262302360,/storyfiles/hs2/01085_2.gif,[002986],002984,,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,storyfiles/hs2/01085_2.gif,storyfiles/hs2/01085_2.gif,\n[EJECT]\n\nhash functions\nA=1; B=3; C=3; D=...


In [517]:
# Time to clean up, now that we know transcripts in either "alt" or "contents_readmspanels"
# Useless columns are "transcriptSrc", "pageNumber_readmspapages", "src_readmspapages", "contents", "data", "src_copy", "src_readmspanels"
# "class", "style" and "tag" should be renamed to "transcript_class", "transcript_style" and "transcript_tag" respectively
# "transcript" should be a new column that contains the value of either "alt" or "contents_readmspanels"
# "alt" and "contents_readmspanels" should be dropped afterwards

# Drop the unnecessary columns
mspa_panels_annotated_df.drop(columns=['transcriptSrc', 'pageNumber_readmspapages', 'src_readmspapages', 'contents', 'data', 'src_copy', 'src_readmspanels'], inplace=True)

# Rename the 'class', 'style', and 'tag' columns
mspa_panels_annotated_df.rename(columns={'class': 'transcript_class', 'style': 'transcript_style', 'tag': 'transcript_tag'}, inplace=True)

# Create the 'transcript' column
mspa_panels_annotated_df['transcript'] = mspa_panels_annotated_df['alt'].fillna(mspa_panels_annotated_df['contents_readmspanels'])

# Drop the 'alt' and 'contents_readmspanels' columns
mspa_panels_annotated_df.drop(columns=['alt', 'contents_readmspanels'], inplace=True)

# Show a panel with cleaned annotations
mspa_panels_annotated_df[mspa_panels_annotated_df['src'] == '/storyfiles/hs2/01085_2.gif']

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,vizStory,vizPageNumber,vizPageId,vizLink,pageNumber,transcript_class,transcript_style,transcript_tag,transcript
3653,story,002985,Dave: Eject your modus and set it to Scrabble ...,You dump all this crap all over the roof.<br /...,1262302360,/storyfiles/hs2/01085_2.gif,[002986],002984,,[],Homestuck,1085,6/1085,https://homestuck.com/story/1085,6/002985,NaN,NaN,NaN,\n[EJECT]\n\nhash functions\nA=1; B=3; C=3; D=...


In [518]:
mspa_panels_annotated_df

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,vizStory,vizPageNumber,vizPageId,vizLink,pageNumber,transcript_class,transcript_style,transcript_tag,transcript
0,story,000006,Look for keyhole,,1180921880,/advimgs/jb/mspaintadventure04.gif,[000008],000005,retro,[],Jailbreak,6,1/6,https://homestuck.com/jailbreak/6,1/000006,NaN,NaN,NaN,NaN
1,story,000006,Look for keyhole,,1180921880,/advimgs/jb/mspaintadventure04b.gif,[000008],000005,retro,[],Jailbreak,6,1/6,https://homestuck.com/jailbreak/6,1/000006,NaN,NaN,NaN,NaN
2,story,000009,Loudly tell that guy to pick up key and try it...,"Despite your bellowing, the man casually opens...",1180931172,/advimgs/jb/mspaintadventure06.gif,[000010],000008,retro,[],Jailbreak,9,1/9,https://homestuck.com/jailbreak/9,1/000009,NaN,NaN,NaN,\nBELLOW!\n\nSLAM\n
3,story,000010,Pee out the window in an attempt to bring him ...,"As you turn around to unzip, you notice anothe...",1180931381,/advimgs/jb/mspaintadventure07.gif,[000011],000009,retro,[],Jailbreak,10,1/10,https://homestuck.com/jailbreak/10,1/000010,NaN,NaN,NaN,NaN
4,story,000010,Pee out the window in an attempt to bring him ...,"As you turn around to unzip, you notice anothe...",1180931381,/advimgs/jb/mspaintadventure07b.gif,[000011],000009,retro,[],Jailbreak,10,1/10,https://homestuck.com/jailbreak/10,1/000010,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11691,psExtras,ps000031,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,/extras/ps000031_3.gif,None,None,None,None,Other Section,ps000031,other/ps000031,https://homestuck.com/story/waywardvagabond/ps...,Unknown/ps000031,NaN,NaN,NaN,NaN
11692,psExtras,ps000040,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,/extras/ps000040_1.gif,None,None,None,None,Other Section,ps000040,other/ps000040,https://homestuck.com/story/waywardvagabond/ps...,Unknown/ps000040,NaN,NaN,NaN,NaN
11693,psExtras,ps000040,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,/extras/ps000040_2.gif,None,None,None,None,Other Section,ps000040,other/ps000040,https://homestuck.com/story/waywardvagabond/ps...,Unknown/ps000040,NaN,NaN,NaN,NaN
11694,psExtras,ps000040,MS Paint Adventures,"\n <tbody><tr>\n <td bgcolor=""#FFFFFF"">\...",None,/extras/ps000040_3.gif,None,None,None,None,Other Section,ps000040,other/ps000040,https://homestuck.com/story/waywardvagabond/ps...,Unknown/ps000040,NaN,NaN,NaN,NaN


# Getting panel tags from HSSE

The Homestuck Search Engine people tagged the tags of the first four acts, from characters to locations and more. Will be extremely useful for image transcription.

In [519]:
"""
HSSE_TAGS and HSSE_SEARCH file contain the entirety of the Homestuck Search Engine tagged data (only the first four acts, until page 1988 inclusive and excluding some swfs) in its own bespoke JSON format.
HSSE_TAGS is the simpler json with tag definitions, and which tags contain other tags:
```json
 "definitions": {
    "0": {
      "_id": 0,
      "name": "Character",
      "children": [
        1,
        32,
        56,
        60,
        104,
        132,
        148,
        155,
        173,
        184,
        253
      ]
    },
    "1": {
      "_id": 1,
      "name": "Human",
      "children": [
        2,
        15
      ]
    },
    "2": {
      "_id": 2,
      "name": "Kid",
      "children": [
        3,
        10
      ]
    },
    "3": {
      "_id": 3,
      "name": "Beta Kid",
      "children": [
        4,
        5,
        7,
        9
      ]
    },
    "4": {
      "_id": 4,
      "name": "John Egbert",
      "children": []
    },
    ...
}
```

HSSE_SEARCH is the more complex json with the actual tags for each panel:
```json
[
  {
    "_id": 0,
    "type": 0,
    "content": "https://www.homestuck.com/images/storyfiles/hs2/00001.gif",
    "thumbnail": "https://www.homestuck.com/images/storyfiles/hs2/00001.gif",
    "url": "https://homestuck.com/story/1",
    "tags": [
      1384,
      1385,
      391,
      321,
      4,
      749,
      801,
      1301,
      602,
      1192,
      711,
      1349
    ],
    "page": 1
  },
  {
    "_id": 1,
    "type": 0,
    "content": "https://www.homestuck.com/images/storyfiles/hs2/00002.gif",
    "thumbnail": "https://www.homestuck.com/images/storyfiles/hs2/00002.gif",
    "url": "https://homestuck.com/story/2",
    "tags": [
      1384,
      1385,
      391,
      321,
      4,
      1349,
      602
    ],
    "page": 2
  },
  ...
]
```
Our objective here is to combine the information so that, for each page, we'll have its human readable tags. 
"""

def load_hsse_tags(tags_path):
    """
    Loads and processes the HSSE tag definitions.

    Args:
        tags_path (str): Path to the hsse_tags.json file.

    Returns:
        dict: A dictionary mapping tag IDs (int) to tag names (str).
    """
    with open(tags_path, 'r', encoding='utf-8') as f:
        tags_data = json.load(f)
    
    # Extract the 'definitions' key
    definitions = tags_data.get('definitions', {})
    
    # Build mapping from _id (int) to name (str)
    tag_id_to_name = {}
    for tag_id_str, tag_info in definitions.items():
        tag_id = tag_info.get('_id')
        name = tag_info.get('name')
        if tag_id is not None and name is not None:
            tag_id_to_name[tag_id] = name
    return tag_id_to_name

def load_hsse_search(search_path):
    """
    Loads and processes the HSSE page-tag associations.

    Args:
        search_path (str): Path to the hsse_search.json file.

    Returns:
        dict: A dictionary mapping page IDs (str) to lists of tag IDs (int).
    """
    with open(search_path, 'r', encoding='utf-8') as f:
        search_data = json.load(f)
    
    # search_data is a list of dicts
    page_to_tag_ids = {}
    for entry in search_data:
        page = entry.get('page')  # integer
        tags = entry.get('tags', [])  # list of integers
        if page is not None:
            page_str = str(page)  # Convert page number to string to match 'pageId'
            page_to_tag_ids[page_str] = tags
    
    return page_to_tag_ids

def map_tags_to_names(page_to_tag_ids, tag_id_to_name):
    """
    Maps tag IDs to tag names for each page.

    Args:
        page_to_tag_ids (dict): Dictionary mapping page IDs to lists of tag IDs.
        tag_id_to_name (dict): Dictionary mapping tag IDs to tag names.

    Returns:
        dict: Dictionary mapping page IDs to lists of tag names.
    """
    page_to_tag_names = {}
    for page_id, tag_ids in page_to_tag_ids.items():
        # Ensure tag_ids is a list
        if not isinstance(tag_ids, list):
            tag_ids = [tag_ids]
        # Map each tag ID to its name, handle unknown tags gracefully
        tag_names = [tag_id_to_name.get(tag_id, f"Unknown Tag {tag_id}") for tag_id in tag_ids]
        page_to_tag_names[page_id] = tag_names
    return page_to_tag_names

def merge_tags_into_mspa_df(mspa_df, page_to_tag_names):
    """
    Merges HSSE tag data into the main MSPA DataFrame.

    Args:
        mspa_df (pd.DataFrame): The main MSPA DataFrame with transcripts.
        page_to_tag_names (dict): Dictionary mapping page IDs to lists of tag names.

    Returns:
        pd.DataFrame: The updated MSPA DataFrame with tags incorporated.
    """
    # Edit page_to_tag_names so the keys have the "6/" prefix to match the Viz page numbers
    page_to_tag_names = {f"6/{k}": v for k, v in page_to_tag_names.items()}
    
    # Create a Series from the page_to_tag_names dictionary
    tags_series = pd.Series(page_to_tag_names, name='tags')
    
    # Merge the tags into mspa_df, matching its vizPageId with the keys of the tags_series
    mspa_df_with_tags = mspa_df.merge(tags_series, left_on='vizPageId', right_index=True, how='left')
    
    # Replace NaN with empty lists for pages without tags
    mspa_df_with_tags['tags'] = mspa_df_with_tags['tags'].apply(lambda x: x if isinstance(x, list) else [])
    
    return mspa_df_with_tags

# Load HSSE tag definitions
tag_id_to_name = load_hsse_tags(HSSE_TAGS)
print(f"Loaded {len(tag_id_to_name)} tags from hsse_tags.json.")

# Load HSSE page-tag associations
page_to_tag_ids = load_hsse_search(HSSE_SEARCH)
print(f"Loaded tag associations for {len(page_to_tag_ids)} pages from hsse_search.json.")

# Map tag IDs to tag names
page_to_tag_names = map_tags_to_names(page_to_tag_ids, tag_id_to_name)
print("Mapped tag IDs to tag names.")

# Merge tags into the main MSPA DataFrame
mspa_df_with_tags = merge_tags_into_mspa_df(mspa_panels_annotated_df, page_to_tag_names)

# # Display the first few rows with tags not an empty list
mspa_df_with_tags[mspa_df_with_tags['tags'].apply(len) > 0].sample(10)

Loaded 1427 tags from hsse_tags.json.
Loaded tag associations for 8002 pages from hsse_search.json.
Mapped tag IDs to tag names.


,section,pageId,title,content,timestamp,src,next,previous,theme,flag,vizStory,vizPageNumber,vizPageId,vizLink,pageNumber,transcript_class,transcript_style,transcript_tag,transcript,tags
4298,story,003501,John: Give dear sweet Casey the bunny.,"<a href=""/archive/external/howdoilive.mp4"" tar...",1269149877,/storyfiles/hs2/01601_2.gif,[003502],003500,,[],Homestuck,1601,6/1601,https://homestuck.com/story/1601,6/003501,NaN,NaN,NaN,NaN,"[Part 1, Act 4, Lined Mode]"
2670,story,002174,Rose: Check status of battery.,"Your LAPTOP BATTERY is alright for now, but it...",1245454297,/storyfiles/hs2/00274_1.gif,[002175],002173,,[],Homestuck,274,6/274,https://homestuck.com/story/274,6/002174,NaN,NaN,NaN,NaN,"[Act 2, Part 1, Mausoleum]"
2517,story,002041,TT: Drop chest.,"|PESTERLOG|<br /><span style=""color: #0715cd"">...",1242114119,/storyfiles/hs2/00141.gif,[002042],002040,,[],Homestuck,141,6/141,https://homestuck.com/story/141,6/002041,NaN,NaN,NaN,NaN,"[Part 1, Act 1, Earth, John's House, Magic Che..."
3823,story,003117,[I] HB: Do a silly dance.,This is just absolutely the most ridiculous th...,1264129922,/storyfiles/hs2/01217.gif,[003118],003116,,[],Homestuck,1217,6/1217,https://homestuck.com/story/1217,6/003117,NaN,NaN,NaN,NaN,"[Intermission, Felt Manor, Hearts Boxcars, HB,..."
4746,story,003846,==>,"A boy finds a dead friend. Her ring, recovered.",1275957781,/storyfiles/hs2/01946_2.gif,[003847],003845,,[R4],Homestuck,1946,6/1946,https://homestuck.com/story/1946,6/003846,NaN,NaN,NaN,NaN,"[Part 1, Act 4, Hero Mode, Ring, Queen's Ring,..."
3795,story,003094,[I] CD: Grab the deuce of clubs.,You pick up two LICORICE GUMMY BEARS.<br /><br...,1263960643,/storyfiles/hs2/01194.gif,[003095],003093,,[],Homestuck,1194,6/1194,https://homestuck.com/story/1194,6/003094,NaN,NaN,NaN,NaN,"[Part 1, Intermission, Clubs Deuce, CD, Gummy ..."
4047,story,003300,==>,"|PESTERLOG|<br /><span style=""color: #e00707"">...",1266291728,/storyfiles/hs2/01400.gif,[003301],003299,,[],Homestuck,1400,6/1400,https://homestuck.com/story/1400,6/003300,NaN,NaN,NaN,NaN,"[Part 1, Act 4, Dave's House, Dave's Room, Dav..."
4741,story,003844,the recent past is recalled...,An ACCURSED MASCOT is located among fallen bre...,1275950569,/storyfiles/hs2/01944_1.gif,[003845],003843,,[],Homestuck,1944,6/1944,https://homestuck.com/story/1944,6/003844,NaN,NaN,NaN,NaN,"[Part 1, Act 4, Imp, Underling, Puppet, Hero M..."
2766,story,002257,Dave: Look out window.,"Yeah, you can kiss all that stuff goodbye. You...",1247185736,/storyfiles/hs2/00357.gif,[002258],002256,,[],Homestuck,357,6/357,https://homestuck.com/story/357,6/002257,NaN,NaN,NaN,OH SNAP,"[Act 2, Part 1, Dave Strider, Record Shirt, Da..."
3390,story,002788,==>,,1258871593,/storyfiles/hs2/00888_2.gif,[002789],002787,,[],Homestuck,888,6/888,https://homestuck.com/story/888,6/002788,NaN,NaN,NaN,NaN,"[Part 1, Act 3, Imp, Dunce Cap, Lined Mode]"


# Extracting character appearances from POV cam 

The POV cam extension for Homestuck allows us to see the characters that are present in each page, and not just until page 1988, all of them. We can use this to extract character tags and somewhat make up for the lack of tags in the later pages.

In [520]:
"""
The data from the POV cam comes in many files named after each character, like "roxy.txt" and "rufioh.txt". The format is not meant to be easily parsable, but it shouldn't be too hard to extract the data and "invert" it, to get the characters that appear in each page and their "commands".

An example of the data (jade.txt):
```
Name: Jade
Colour: #4AC925
Image: jade.png
Group: Kids

Be created on meteor
3790-3791
3803
3807
3830-3831

Be sent to Earth
3840

Land on factory
3768-3769

Be adopted
3773-3775

Be taken on hunt with grandfather
Wander off with Bec
Find present
3029-3036
```
From the documentation:
```md
## Timeline language

In the `Readable Timelines` folder are a number of files, each containing the timeline data for a single person.

The files use the following format:

 * Page numbers or ranges of numbers to describe what pages a person's on.
   (For A6A5A1x2 COMBO, use `-2` on the end to go through the pages two at a time)
   eg. `6009`, `1901-2032`, or `7688-7692-2`
 * To split the timeline, indent the splintered timeline, then return to the original indentation for the alpha timeline.
   Note that the two timelines are not connected by default, you must use the next two commands to describe how they should be joined.
 * `==>`: Jump into the next split timeline from this point
 * `<==`: Jump out of previous split timeline to this point
 * `~`: Insert another timeline here, using a person's name.
   eg. `~ Davesprite`

The following commands change properties about the current person or timeline.
Write the exact word, then `:`, then the value you wish to set it to.
eg. `Name: John`.

 * `Name`: Change the name of the current person.
 * `Colour`: Change the colour used for the links.
 * `Image`: Change the image used for the links.
 * `Group`: Change which group the links are a part of.
 * `Caption`: Give some hover-over text to the link between the previous page and the next.

All lines which do not fit any of the above are ignored, like comments.
```
"""


def extract_data_from_pov_cam_file(pov_cam_file: str) -> pd.DataFrame:
    """
    Extracts data from a POV cam file.

    Args:
        pov_cam_file (str): The path to the POV cam file.

    Returns:
        pd.DataFrame: A DataFrame containing the extracted data with columns ['page', 'character', 'commands'].
    """
    # Regular expression to identify page lines (single number or range with optional step)
    page_pattern = re.compile(r'^\d+(-\d+)?(-\d+)?$')

    try:
        with open(pov_cam_file, 'r', encoding='utf-8') as file:
            lines = file.readlines()
    except FileNotFoundError:
        print(f"Error: File not found - {pov_cam_file}")
        return pd.DataFrame(columns=['page', 'character', 'commands'])
    except Exception as e:
        print(f"Error reading file {pov_cam_file}: {e}")
        return pd.DataFrame(columns=['page', 'character', 'commands'])

    # Initialize the data storage
    data = {
        'page': [],
        'character': [],
        'commands': []
    }

    # Initialize current character and commands
    current_character = None
    current_commands = []
    last_commands = []

    for line_number, line in enumerate(lines, start=1):
        original_line = line  # Keep the original line for debugging if needed
        line = line.strip()

        if not line:
            continue  # Skip empty lines

        # Check for metadata lines
        if line.startswith("Name: "):
            # Extract character name from the 'Name:' line
            extracted_name = line.split("Name: ", 1)[1].strip()
            if current_character is None:
                current_character = extracted_name
            else:
                # Ignore subsequent 'Name:' lines to prevent name changes from affecting the character name
                continue
            current_commands = []  # Reset commands when a new character is found
            last_commands = []     # Reset last_commands for the new character
            continue  # Move to the next line
        elif any(line.startswith(prefix) for prefix in ["Colour: ", "Image: ", "Group: "]):
            continue  # Skip other metadata lines

        # Check if the line is a page line
        if page_pattern.match(line):
            if current_commands:
                # If there are new commands, update last_commands
                last_commands = current_commands.copy()
                commands_to_use = current_commands.copy()
            elif last_commands:
                # If no new commands, inherit last_commands
                commands_to_use = last_commands.copy()
            else:
                # If no commands to inherit, assign ["==>"]
                commands_to_use = ["==>"]

            # Parse the page or range
            pages = []
            parts = line.split('-')
            try:
                if len(parts) == 1:
                    pages = [int(parts[0])]
                elif len(parts) == 2:
                    start, end = map(int, parts)
                    pages = list(range(start, end + 1))
                elif len(parts) == 3:
                    start, end, step = map(int, parts)
                    pages = list(range(start, end + 1, step))
                else:
                    print(f"Warning: Unexpected page format at line {line_number}: '{original_line}'")
                    continue  # Skip unexpected formats
            except ValueError:
                print(f"Error: Non-integer page numbers at line {line_number}: '{original_line}'")
                continue  # Skip lines with non-integer values

            for page in pages:
                if page in data['page']:
                    # If the page already exists, append the new commands without duplication
                    index = data['page'].index(page)
                    existing_commands = data['commands'][index]
                    for cmd in commands_to_use:
                        if cmd not in existing_commands:
                            existing_commands.append(cmd)
                else:
                    data['page'].append(page)
                    data['character'].append(current_character)
                    data['commands'].append(commands_to_use.copy())  # Use a copy to prevent mutation

            current_commands = []  # Reset commands after assigning them to pages
        else:
            # Assume it's a command line
            current_commands.append(line)

    # Remove duplicate commands per page while preserving order
    for i, cmds in enumerate(data['commands']):
        seen = set()
        unique_cmds = []
        for cmd in cmds:
            if cmd not in seen:
                unique_cmds.append(cmd)
                seen.add(cmd)
        data['commands'][i] = unique_cmds

    # Create DataFrame
    df = pd.DataFrame(data)
    return df


def extract_pov_cam_data(pov_cam_folder: str) -> pd.DataFrame:
    """
    Extracts data from all POV cam files in a given folder.

    Args:
        pov_cam_folder (str): The path to the folder containing POV cam text files.

    Returns:
        pd.DataFrame: A combined DataFrame containing data from all POV cam files.
    """
    # List all .txt files in the folder
    pov_cam_files = [file for file in os.listdir(pov_cam_folder) if file.endswith('.txt')]
    pov_cam_file_with_full_path = [os.path.join(pov_cam_folder, file) for file in pov_cam_files]

    # Initialize a list to hold DataFrames from each file
    pov_cam_data_frames = []

    for file_path in pov_cam_file_with_full_path:
        df = extract_data_from_pov_cam_file(file_path)
        if not df.empty:
            pov_cam_data_frames.append(df)
        else:
            print(f"No data extracted from {file_path}")

    if pov_cam_data_frames:
        pov_cam_data = pd.concat(pov_cam_data_frames, ignore_index=True)
    else:
        pov_cam_data = pd.DataFrame(columns=['page', 'character', 'commands'])

    return pov_cam_data


# Extract data from the POV cam files
pov_cam_data = extract_pov_cam_data(POV_CAM_FOLDER)

# Display the extracted data
pov_cam_data.sample(10)

,page,character,commands
9853,7974,John,"[Find treasure, Cut away from reveal]"
5573,4743,Jack Noir,[Take Bro's sword from Beat Mesa]
4096,5707,Doc Scratch,[The dÎsc is tø€ badly da™aged. You can no lcœ...
3336,3559,Davesprite,"[Scold Terezi, Talk to Dave]"
9116,3478,John,[Be trolled by Terezi]
17278,4169,Vriska,[Put lusus out of misery]
10978,10012,Kanaya,"[Drag Rose to Dave, Karkat]"
17560,9267,Vriska,"[FL1P, John????????, Be knocked out]"
4277,4961,Draconian Dignitary,[Be encouraged to make Becquerel]
12173,6900,Meenah,"[Be in fight with Dave, Karkat, Kanaya, Interr..."


In [521]:
# Show some unique complex data, like pages with multiple commands
pov_cam_data[pov_cam_data['commands'].apply(len) > 2].sample(10)

,page,character,commands
13644,4586,Rose,"[Talk with John about building his house, wrea..."
12410,9755,Meenah,"[Debrief Vriska, Fail to come up with plan, TA..."
1330,7569,Caliborn,"[Jeer Jake, Give Jake uROBuROS code, Possess L..."
1150,7874,Caliborn,"[Land on Earth, Be found by Gamzee, Hatch, Sep..."
1981,8097,Cans,"[Become minion of Caliborn, Witness intense st..."
10132,8848,John,"[Fly around, Find Dirk, Follow Roxy, Rose to L..."
14720,8881,Roxy,"[Carry Rose to LOPAN, Watch Rose bleed out, Ta..."
14768,9008,Roxy,"[Be on LOWAS, sup, Take Dave's bro's sword]"
1190,7077,Caliborn,"[<==, Jeer Calliope about death of Jane, Compl..."
13204,2068,Rose,"[Connect to John, Mess around with interface, ..."


In [522]:
# Now that we have the page/character/commands dataset, we can merge the page/character data
# We'll turn the commands into an array of commands prefixed with the character name
# We'll also add a "pov_characters" and "pov_commands" column to the mspa_df
# Note that for the POV cam, the vizPageId is actually incorrect. We can use the actual page number in pageId instead,
# though it's six padded digits in mspa_df_with_tags and an actual integer in pov_cam_data. We'll padd the latter to match.

# Pad the page numbers in pov_cam_data
pov_cam_data['pageId'] = pov_cam_data['page'].apply(lambda x: f"{x:06d}")

# Merging the data now would be silly, we need to create a new dataframe where there's only one row per page
# We'll concatenate the commands for each character in the same page
# We'll also add the commands to a list, and the characters to a list

# Add the character names to the commands if the commands don't already have them
pov_cam_data['commands'] = pov_cam_data.apply(lambda x: [f"{x['character']}: {cmd}" for cmd in x['commands']], axis=1)

# Group by pageId and character
grouped = pov_cam_data.groupby(['pageId']).agg({
    'character': list,
    'commands': list
}).reset_index()

grouped.sample(10)

,pageId,character,commands
6832,009046,[John],[[John: <==]]
1650,003562,[John],[[John: <==]]
2896,004873,"[John, Vriska]","[[John: Talk with Vriska, John: WHY DON'T YOU ..."
6770,008984,"[Calliope, Jade]","[[Calliope: Start drawing story, Calliope: Be ..."
2233,004172,[Vriska],[[Vriska: Put lusus out of misery]]
1752,003678,[Authority Regulator],[[Authority Regulator: Follow Lil' Cal on hove...
2584,004560,[Warweary Villein],[[Warweary Villein: Find ring]]
2934,004911,"[Becquerel, Jade, Parcel Mistress]","[[Becquerel: Destroy uranium imp], [Jade: Stri..."
1074,002982,"[Dave, Lil' Cal]","[[Dave: Mourn loss of Lil' Cal], [Lil' Cal: Be..."
6218,008425,"[Caliborn, John]","[[Caliborn: Continue Homosuck, Caliborn: Be in..."


In [523]:
# Merge the data into mspa_df_with_tags
# "character" can become "pov_characters" and "commands" can become "pov_commands"

# rename columns
grouped.rename(columns={'character': 'pov_characters', 'commands': 'pov_commands'}, inplace=True)

mspa_df_with_pov = mspa_df_with_tags.merge(grouped, on='pageId', how='left')

# Replace NaN with empty lists for pages without POV cam data
mspa_df_with_pov['pov_characters'] = mspa_df_with_pov['pov_characters'].apply(lambda x: x if isinstance(x, list) else [])
mspa_df_with_pov['pov_commands'] = mspa_df_with_pov['pov_commands'].apply(lambda x: x if isinstance(x, list) else [])

mspa_df_with_pov[mspa_df_with_pov['pov_characters'].apply(len) > 0].sample(10) 

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,...,vizPageId,vizLink,pageNumber,transcript_class,transcript_style,transcript_tag,transcript,tags,pov_characters,pov_commands
2514,story,002044,TT: Select stuff in yard and move it back into...,"|PESTERLOG|<br /><span style=""color: #0715cd"">...",1242181111,/storyfiles/hs2/00144.gif,[002045],002043,,[],...,6/144,https://homestuck.com/story/144,6/002044,NaN,NaN,NaN,NaN,"[Part 1, Act 1, Earth, Sburb, Sburb Logo, Sbur...","[John, Rose]","[[John: Get trapped in room], [Rose: Connect t..."
3438,story,002830,==>,You zoom out.<br /><br />It looks like you and...,1259529123,/storyfiles/hs2/00930.gif,[002831],002829,,[],...,6/930,https://homestuck.com/story/930,6/002830,NaN,NaN,NaN,-3140D:T16:13:00,"[Part 1, Act 3, Jaspers, Screen, Rose Lalonde,...",[Rose],"[[Rose: Be told Jasper's secret, Rose: Find ap..."
2778,story,002269,==>,"But part of it was touching the floor, so your...",1247431736,/storyfiles/hs2/00369.gif,[002270],002268,,[],...,6/369,https://homestuck.com/story/369,6/002269,NaN,NaN,NaN,NaN,"[Act 2, Part 1, Rose's House, Fridge, Kitchen,...",[Rose],[[Rose: Head outside]]
3133,story,002569,WV: Check the little red bar.,It appears to be a gauge for a large POWER CEL...,1253594874,/storyfiles/hs2/00669.gif,[002570],002568,,[],...,6/669,https://homestuck.com/story/669,6/002569,NaN,NaN,NaN,NaN,"[Part 1, Act 2, WV, Exile, Carapacian, Dersite...",[Warweary Villein],"[[Warweary Villein: WV: Retri..., Ascend.]]"
4766,story,003853,==>,"The mail is delivered. An obligation, satisfied.",1276041795,/storyfiles/hs2/01953_2.gif,[003854],003852,,[],...,6/1953,https://homestuck.com/story/1953,6/003853,NaN,NaN,NaN,NaN,"[Part 1, Act 4, Carapacian, PM, Weapon, Sword,...","[Jack Noir, John, Parcel Mistress]","[[Jack Noir: Exchange with PM, gift to John], ..."
4451,story,003631,==>,You settle in front of a man in sore need of a...,1271987780,/storyfiles/hs2/01731.gif,[003632],003630,,[],...,6/1731,https://homestuck.com/story/1731,6/003631,NaN,NaN,NaN,NaN,"[Part 1, Act 4, Dad Egbert, Dad, LOWAS, Dad's ...",[Dad],"[[Dad: Receive hat, Dad: Meet Grandpa, Dad: Te..."
9881,story,008448,[A6A6I2] ====>,,1378186130,/storyfiles/hs2/06545.gif,[008449],008447,,[],...,6/6548,https://homestuck.com/story/6548,6/008448,NaN,NaN,NaN,NaN,[],"[Aranea, Gamzee]","[[Aranea: Mind control Gamzee to get ring, Ara..."
5306,story,004294,Examine flushed quadrant.,When two individuals find themselves in the fl...,1281484793,/storyfiles/hs2/02394_1.gif,[004295],004293,,[],...,6/2394,https://homestuck.com/story/2394,6/004294,NaN,NaN,NaN,NaN,[],"[Dad, Mom]","[[Dad: Stand on helm, armed with cake], [Mom: ..."
5817,story,004772,==>,,1289804828,/storyfiles/hs2/02871.gif,[004773],004771,,[],...,6/2872,https://homestuck.com/story/2872,6/004772,NaN,NaN,NaN,NaN,[],"[Bro, Jack Noir]","[[Bro: Duel with Jack Noir], [Jack Noir: Retur..."
10133,story,008694,[A6A6I2] ====>,,1381111904,/storyfiles/hs2/06791.gif,[008695],008693,,[],...,6/6794,https://homestuck.com/story/6794,6/008694,NaN,NaN,NaN,\nPOW\n\ncavort\n,[],"[Gamzee, Terezi]","[[Gamzee: Be attacked by Terezi], [Terezi: Fig..."


# Adding Acts and Pages of Homestuck data

I completely forgot this existed until I was already trying to finetune a model, but this is critical information. For each act and even groups of pages, we have detailed synopses. We'll actually need to crawl and generate the dataset ourselves.

In [524]:
"""
```html
<div>
<link rel="stylesheet" type="text/css" href="style-nav.css">

<table id="sidebar-nav">
<tbody><tr>
<th colspan="4" style="font-size: larger;">
<p><a href="/homestuck/"><img alt="Main Page" src="/homestuck/sburb-in-spiro.png" height="200"></a></p>
<p style="margin-top: 0.3em; font-size: 80%;">the Acts and Pages of</p>
<p style="margin-top: 0.1em; font-size: 140%;"><a href="https://homestuck.com/story/">HOMESTUCK</a></p>
<p style="margin: 0.5em 0.2em; font-size: 90%; line-height: 0.95em;">a guide for fans of<br>the web adventure</p>
</th>
</tr>
<tr></tr>
<tr>
<td><strong><img src="/homestuck/map/a1.png" alt=""><br>Act 1</strong></td>
<td><a href="/homestuck/act-2"><img src="/homestuck/map/a2.png" alt=""><br>Act 2</a></td>
<td><a href="/homestuck/act-3"><img src="/homestuck/map/a3.png" alt=""><br>Act 3</a></td>
<td><a href="/homestuck/intermission"><img src="/homestuck/map/int.png" alt=""><br>Int</a></td>
</tr>
<tr>
<td><a href="/homestuck/act-4"><img src="/homestuck/map/a4.png" alt=""><br>Act 4</a></td>
<td><a href="/homestuck/a5a1"><img src="/homestuck/map/a5a1.png" alt=""><br>5–1</a></td>
<td><a href="/homestuck/a5a2"><img src="/homestuck/map/a5a2.png" alt=""><br>5–2</a></td>
<td class="scaps"><a href="/homestuck/eoa5"><img src="/homestuck/map/cascade.png" alt=""><br>EOA5</a></td>
</tr>
<tr>
<td><a href="/homestuck/a6-1"><img src="/homestuck/map/a6a1.png" alt=""><br>6–1</a></td>
<td><a href="/homestuck/a6-2"><img src="/homestuck/map/a6a2.png" alt=""><br>6–2</a></td>
<td><a href="/homestuck/a6-3"><img src="/homestuck/map/a6a3.png" alt=""><br>6–3</a></td>
<td><a href="/homestuck/a6-4"><img src="/homestuck/map/a6a4.png" alt=""><br>6–4</a></td>
</tr>
<tr>
<td><a href="/homestuck/a6-5"><img src="/homestuck/map/a6a5.png" alt=""><br>6–5</a></td>
<td><a href="/homestuck/a6a6-1"><img src="/homestuck/map/a6a6a1-hi.png" alt=""><br>6-6-1</a></td>
<td><a href="/homestuck/a6a6-2"><img src="/homestuck/map/a6a6a2-hi.png" alt=""><br>6-6-2</a></td>
<td><a href="/homestuck/a6a6-3"><img src="/homestuck/map/a6a6a3-hi.png" alt=""><br>6-6-3</a></td>
</tr>
<tr>
<td><a href="/homestuck/a6a6-4"><img src="/homestuck/map/a6a6a4-hi.png" alt=""><br>6-6-4</a></td>
<td><a href="/homestuck/a6a6-5"><img src="/homestuck/map/a6a6a5-hi.png" alt=""><br>6-6-5</a></td>
<td><a href="/homestuck/a6a6a6"><img src="/homestuck/map/a6a6a6-hi.png" alt=""><br>6-6-6</a></td>
<td><a href="/homestuck/act-7"><img src="/homestuck/map/a7.png" alt=""><br>Act 7</a></td>
</tr>
<tr>
<td><a href="/homestuck/part-1"><img src="/homestuck/sburb.png" alt=""><br>Part 1</a></td>
<td><a href="/homestuck/part-2"><img src="/homestuck/sgrub.png" alt=""><br>Part 2</a></td>
<td><a href="/homestuck/part-3"><img src="/homestuck/sburb-a.png" alt=""><br>Part 3</a></td>
<td><a href="/homestuck/part-4"><img src="/homestuck/sburb-r-small.png" alt=""><br>Part 4</a></td>
</tr>
<tr></tr>
<tr>
<th colspan="4">
<p style="font-size: 120%; line-height: 1.1em; margin-top: 0.2em;">documented by</p>
<p style="font-size: 140%; line-height: 1em;">Rafe Saltman</p>
<p style="font-size: 80%;"><a href="mailto:rafe@rafe.name">email</a> &nbsp;<a href="https://rafr.tumblr.com">tumblr</a>&nbsp; <a href="https://twitter.com/Rafes">twitter</a></p>
<p style="font-size: 120%; line-height: 1em; margin-top: 1.2em;">art and excerpts<br>from <i>Homestuck</i></p>
<p style="font-size: 120%; line-height: 1.5em;"><img src="/homestuck/mspa-face.png" class="face"> © 2009-2016 <img src="/homestuck/mspa-flip.png" class="face"></p>
<p style="font-size: 140%">Andrew Hussie</p>
</th>
</tr>
</tbody></table>

<table id="header-nav">
<tbody><tr>
<th rowspan="6">
<p><a href="/homestuck/"><img alt="Main Page" src="/homestuck/sburb-in-spiro.png" height="175"></a></p>
<p style="margin-top: 0.6em; font-size: 85%;">the Acts and Pages of</p>
<p style="font-size: 150%;"><a href="https://homestuck.com/story/">HOMESTUCK</a></p>
<p style="margin-top: 0.5em; font-size: 95%; line-height: 1em;">a guide for fans of<br>the web adventure</p>
<p style="line-height: 1.1em; margin-top: 1.5em;">documented by</p>
<p style="font-size: 115%;">Rafe Saltman</p>
<p style="font-size: 72%;"><a href="mailto:rafe@rafe.name">email</a>&nbsp; <a href="https://rafr.tumblr.com">tumblr</a> &nbsp;<a href="https://twitter.com/Rafes">twitter</a></p>
<p style="line-height: 1.1em; margin-top: 1.5em;">art and excerpts<br>from <i>Homestuck</i></p>
<p><img src="/homestuck/mspa-face.png" class="face"> © 2009-2016 <img src="/homestuck/mspa-flip.png" class="face"></p>
<p style="font-size: 115%;">Andrew Hussie</p>
</th>
<td><strong><img src="/homestuck/map/a1.png" alt=""><br>Act 1</strong></td>
<td><a href="/homestuck/act-2"><img src="/homestuck/map/a2.png" alt=""><br>Act 2</a></td>
<td><a href="/homestuck/act-3"><img src="/homestuck/map/a3.png" alt=""><br>Act 3</a></td>
<td><a href="/homestuck/intermission"><img src="/homestuck/map/int.png" alt=""><br>Int</a></td>
</tr>
<tr>
<td><a href="/homestuck/act-4"><img src="/homestuck/map/a4.png" alt=""><br>Act 4</a></td>
<td><a href="/homestuck/a5a1"><img src="/homestuck/map/a5a1.png" alt=""><br>5–1</a></td>
<td><a href="/homestuck/a5a2"><img src="/homestuck/map/a5a2.png" alt=""><br>5–2</a></td>
<td class="scaps"><a href="/homestuck/eoa5"><img src="/homestuck/map/cascade.png" alt=""><br>EOA5</a></td>
</tr>
<tr>
<td><a href="/homestuck/a6-1"><img src="/homestuck/map/a6a1.png" alt=""><br>6–1</a></td>
<td><a href="/homestuck/a6-2"><img src="/homestuck/map/a6a2.png" alt=""><br>6–2</a></td>
<td><a href="/homestuck/a6-3"><img src="/homestuck/map/a6a3.png" alt=""><br>6–3</a></td>
<td><a href="/homestuck/a6-4"><img src="/homestuck/map/a6a4.png" alt=""><br>6–4</a></td>
</tr>
<tr>
<td><a href="/homestuck/a6-5"><img src="/homestuck/map/a6a5.png" alt=""><br>6–5</a></td>
<td><a href="/homestuck/a6a6-1"><img src="/homestuck/map/a6a6a1-hi.png" alt=""><br>6-6-1</a></td>
<td><a href="/homestuck/a6a6-2"><img src="/homestuck/map/a6a6a2-hi.png" alt=""><br>6-6-2</a></td>
<td><a href="/homestuck/a6a6-3"><img src="/homestuck/map/a6a6a3-hi.png" alt=""><br>6-6-3</a></td>
</tr>
<tr>
<td><a href="/homestuck/a6a6-4"><img src="/homestuck/map/a6a6a4-hi.png" alt=""><br>6-6-4</a></td>
<td><a href="/homestuck/a6a6-5"><img src="/homestuck/map/a6a6a5-hi.png" alt=""><br>6-6-5</a></td>
<td><a href="/homestuck/a6a6a6"><img src="/homestuck/map/a6a6a6-hi.png" alt=""><br>6-6-6</a></td>
<td><a href="/homestuck/act-7"><img src="/homestuck/map/a7.png" alt=""><br>Act 7</a></td>
</tr>
<tr>

<td style="vertical-align: bottom;"><a href="/homestuck/part-1"><img src="/homestuck/sburb.png" alt=""><br>Part 1</a></td>
<td style="vertical-align: bottom;"><a href="/homestuck/part-2"><img src="/homestuck/sgrub.png" alt=""><br>Part 2</a></td>
<td style="vertical-align: bottom;"><a href="/homestuck/part-3"><img src="/homestuck/sburb-a.png" alt=""><br>Part 3</a></td>
<td style="vertical-align: bottom;"><a href="/homestuck/part-4"><img src="/homestuck/sburb-r-small.png" alt=""><br>Part 4</a></td>
</tr>
</tbody></table>
<table>
<tbody><tr>
<th style="width: 160px"><a href="#a1-0"><img src="map/a1-hi.png" alt="Act 1 thumbnail" height="148px"></a></th>
<th colspan="3">
<h1 class="tint">Act 1</h1>
<h1 class="title">The Note Desolation Plays</h1>
<h4 class="spoilers">Spoilers ahead!</h4></th>
</tr>
<tr class="stats">
<th>first page</th>
<td><a href="/hs/1">1</a>, April 13 2009</td>
<th>pages</th>
<td>247</td>
</tr>
<tr class="stats">
<th>last page</th>
<td><a href="/hs/247">247</a>, June 7 2009</td>
<th>words in text</th>
<td id="words-text">10 563</td>
</tr>
<tr class="stats">
<th>recap</th>
<td><a href="/hs/1674">1674</a>, April 15 2010</td>
<th><a href="http://readmspa.org/search/">words in media</a></th>
<td id="words-media">2032</td>
</tr>
<tr><th></th><th colspan="3" style="padding-top: 1em;">Synopsis</th></tr>
<tr class="synopsis"><th>john</th>
<td colspan="3">
<p>Homestuck begins as the story of four friends who live far apart and have never met. We are introduced to these kids one by one, starting with John Egbert.</p>
<p>It is John’s thirteenth birthday, and he wants to play a game with his online friends. He has been waiting for the game to arrive in the mail for days. After fussing with a gift from his baking-crazed Dad and enduring the pesterings of his friends, John leaves his room to check the mailbox.</p>
<p>Seeing that Dad has already brought in the mail, John confronts him in the kitchen. They have a messy battle with hammers and cakes and pies. John ends the strife with a distraction and makes off with his mail.</p>
<p>There are two parcels addressed to himself. The first, a gift from his friend Dave, contains the plush bunny prop from the set of <i>Con Air</i>. The second contains the beta release of the computer game Sburb. John and his friend Rose are eager to play Sburb and find out what it is about.</p>
<p>Two other parcels for John remain out of reach, locked in Dad’s car.</p>
</td></tr>
<tr class="synopsis"><th>sburb</th>
<td colspan="3">
<p>John starts a game of Sburb with Rose.</p>
<p>As John’s server player, Rose assumes control of his environment. Experimenting with the controls on her end, Rose discovers that she can extend John’s house, move furniture around, and place machinery that supports a kind of alchemy based on punch cards.</p>
<p>The alchemy equipment emits a confusing ball of light called the Kernelsprite. The sprite is “prototyped” when it absorbs John’s maimed harlequin doll and takes on the appearance of a one-eyed, one-armed jester.</p>
<p>The mission to alchemize their first item becomes urgent when John discovers a meteor is about to hit his house. After a series of frustrations brought on by Rose’s spotty internet connection, they succeed in creating a tree that bears a blue apple. John has only enough time to take a bite of the apple before the meteor strikes the Earth and makes a crater where his house stood a moment before.</p>
</td></tr>
</tbody></table>
<table>
<tbody><tr>
<th colspan="4">Act 1 by date posted</th>
</tr>
<tr></tr>
<tr>
<th>date</th><th>page</th><th>scenes and events</th><th>adventure log</th>
</tr>
<tr id="a1-0">
<td>April 13 2009</td>
<td>1</td>
<td>
<p><img src="map/a1.png" height="74" alt=""></p>
<p><strong class="caps">Act 1.</strong></p>
<p>Who is this young man?</p>
<p>John Egbert examines his room.</p>
</td>
<td class="log">
<p id="1"><a href="/hs/1">Homestuck</a></p>
<p id="2"><a href="/hs/2">Enter name.</a></p>
<p id="3"><a href="/hs/3">Try again.</a></p>
<p id="4"><a href="/hs/4">Examine room.</a></p>
<p id="5"><a href="/hs/5">John: Quickly retrieve arms from drawer.</a></p>
<p id="6"><a href="/hs/6">Remove CAKE from MAGIC CHEST.</a></p>
<p id="7"><a href="/hs/7">John: Quickly retrieve arms from MAGIC CHEST.</a></p>
<p id="8"><a href="/hs/8">John: Examine contents of chest.</a></p>
<p id="9"><a href="/hs/9">John: Captchalogue smoke pellets.</a></p>
<p id="10"><a href="/hs/10">John: Equip fake arms.</a></p>
<p id="11"><a href="/hs/11">John: Examine Problem Sleuth Poster.</a></p>
<p id="12"><a href="/hs/12">John: Read note on drawer.</a></p>
<p id="13"><a href="/hs/13">John: Take poster.</a></p>
<p id="14"><a href="/hs/14">John: Acquire hammer and nails. They will come in handy.</a></p>
<p id="15"><a href="/hs/15">John: Take nails.</a></p>
<p id="16"><a href="/hs/16">John: Squawk like an imbecile and shit on your desk.</a></p>
</td>
</tr>
<tr>
<td>April 14 2009</td>
<td>17</td>
<td>
<p>John puts up his new Little Monsters poster.</p>
</td>
<td class="log">
<p id="17"><a href="/hs/17">John: Combine the nails and hammer.</a></p>
<p id="18"><a href="/hs/18">John: Use hammer/nails on poster.</a></p>
<p id="19"><a href="/hs/19">John: Nail poster to wall.</a></p>
<p id="20"><a href="/hs/20">John: Examine Con Air poster.</a></p>
<p id="21"><a href="/hs/21">John: Examine Deep Impact poster.</a></p>
<p id="22"><a href="/hs/22">John: Examine calendar.</a></p>
<p id="23"><a href="/hs/23">John: Eat cake.</a></p>
</td>
</tr>
...
```

The above is an excerpt of the HTML in one of the pages of the website, specifically https://rafe.name/homestuck/act-1.
We'll obviously have to crawl all the act pages, and the part pages. Each part contains multiple acts, each act contains multiple groups of pages. 
We'll end up with a dataset (after exploding the group-based dataset) like...
page_id | page_group_summary | act | act_summary | part | part_summary
1 | "John Egbert examines his room." | "Act 1" | [["JOHN", "Homestuck begins as the story of four friends who live far apart and have never met. We are introduced to these kids one by one, starting with John Egbert...."], ["SBURB", "The alchemy equipment emits a confusing ball of light called the Kernelsprite. The sprite is “prototyped” when it absorbs John’s maimed harlequin doll and takes on the appearance of a one-eyed, one-armed jester..."]] | "Part 1" | "John, Rose, Dave, and Jade are online friends. On John’s thirteenth birthday, the kids play a game that takes them and their guardians to a set of fantastic worlds called the Medium. They are challenged to work together, kill monsters, learn punch-card alchemy, and build their houses up to the heavenly orb of Skaia..."

This should be really useful when we're trying to guess the tags that are in a page. The summaries should also be generally useful for text-based models later on. 
"""




'\n```html\n<div>\n<link rel="stylesheet" type="text/css" href="style-nav.css">\n\n<table id="sidebar-nav">\n<tbody><tr>\n<th colspan="4" style="font-size: larger;">\n<p><a href="/homestuck/"><img alt="Main Page" src="/homestuck/sburb-in-spiro.png" height="200"></a></p>\n<p style="margin-top: 0.3em; font-size: 80%;">the Acts and Pages of</p>\n<p style="margin-top: 0.1em; font-size: 140%;"><a href="https://homestuck.com/story/">HOMESTUCK</a></p>\n<p style="margin: 0.5em 0.2em; font-size: 90%; line-height: 0.95em;">a guide for fans of<br>the web adventure</p>\n</th>\n</tr>\n<tr></tr>\n<tr>\n<td><strong><img src="/homestuck/map/a1.png" alt=""><br>Act 1</strong></td>\n<td><a href="/homestuck/act-2"><img src="/homestuck/map/a2.png" alt=""><br>Act 2</a></td>\n<td><a href="/homestuck/act-3"><img src="/homestuck/map/a3.png" alt=""><br>Act 3</a></td>\n<td><a href="/homestuck/intermission"><img src="/homestuck/map/int.png" alt=""><br>Int</a></td>\n</tr>\n<tr>\n<td><a href="/homestuck/act-4"><img

# Adding MSPA commentary data

From the books, we have commentary for groups of pages in Problem Sleuth and Homestuck that might be a bit pointless for prediction models, but is still useful text data.

In [525]:
"""
MSPA_COMMENTARY is a json export of an SQL table that contains the per-Viz-page commentary (by the creator Andrew Hussie) and notes (by me and other volunteers, like fanmade commentary).
There are two tables, one for Homestuck ("commentary") and another for Problem Sleuth ("problem-sleuth") with this format:
```json
[
{
    "type": "table",
    "name": "story",
    "database": "commentary",
    "data": 
[
    {
        "id":"1",
        "title":"Homestuck",
        "page":"7",
        "commentary":"Homestuck (the stable release) began on April 13th (i.e. 4\/13), 2009. My intent was always to make the start date a very significant number in the story, recurring frequently. Consequently, I decided to make him thirteen years old, thus making the story about four thirteen-year-old kids. There are more references to this number than can be mentioned casually, some of which are serendipitous. Playing cards, which have a good deal of relevance later, are comprised of four suits of thirteen cards each, for instance.",
        "notes":null
     },
...
{
    "type": "table",
    "name": "problem-sleuth",
    "database": "commentary",
    "data": 
[
{
    "id": "1",
    "title": "Problem Sleuth",
    "page": "15",
    "commentary": "The title \"Problem Sleuth\" resulted from a quest to determine the most awkward and peculiar term for a detective imaginable. I ran a bunch of other possibilities by my friend, and she instantly green-lighted PS when I said it. Little did I know I would find a way to top that phrase when introducing the tall skinny one a few hundred pages later. Though his name probably wouldn't have made as good a title, admittedly.",
    "notes": null
},
...
```
It should be fairly trivial to turn this into a dataset that we can merge based on the adventure title ("vizStory") and page number ("id"->"vizPageNumber", the "page" is the physical book page). Then, we can keep the page_commentary and page_notes columns.
"""

# Load the data
with open(MSPA_COMMENTARY, 'r', encoding="utf-8") as f:
    commentary_data = json.load(f)

hs_data = commentary_data[0]['data']
ps_data = commentary_data[1]['data']

# Create DataFrames
hs_df = pd.DataFrame(hs_data)
ps_df = pd.DataFrame(ps_data)

# Fill the vizStory column with each title on each dataframe
hs_df['vizStory'] = 'Homestuck'
ps_df['vizStory'] = 'Problem Sleuth'
    
# Concatenate the DataFrames
commentary_df = pd.concat([hs_df, ps_df], ignore_index=True)

# We don't care about the title or the page (book page), we care about id (renamed to vizPageNumber), the commentary and notes
commentary_df = commentary_df[['vizStory', 'id', 'commentary', 'notes']]
commentary_df.rename(columns={'id': 'vizPageNumber', 'commentary': 'pageCommentary', 'notes': 'pageNotes'}, inplace=True)

# Show some data
commentary_df.sample(10)

,vizStory,vizPageNumber,pageCommentary,pageNotes
5014,Problem Sleuth,1302,Off-camera bird holding pryglass: Lower Prygla...,None
438,Homestuck,439,His finger slips and unlocks the CAPS LOCK key...,None
3010,Homestuck,3012,None,None
4931,Problem Sleuth,1219,None,None
4147,Problem Sleuth,435,None,None
4621,Problem Sleuth,909,This is also one of the few instances where th...,None
1582,Homestuck,1584,None,None
5321,Problem Sleuth,1609,None,None
871,Homestuck,873,None,None
3526,Homestuck,3528,Aside from being a great series of panels here...,None


In [526]:
# We know vizStory "Homestuck" and vizPage "3052" has commentary, print the record in each dataframe to confirm the merge will work
print(commentary_df[(commentary_df['vizStory'] == 'Homestuck') & (commentary_df['vizPageNumber'] == '3052')])
print(mspa_df_with_pov[(mspa_df_with_pov['vizStory'] == 'Homestuck') & (mspa_df_with_pov['vizPageNumber'] == 3052)])

# We need to convert mspa_df_with_pov['vizPageNumber'] to a string to merge
mspa_df_with_pov['vizPageNumber'] = mspa_df_with_pov['vizPageNumber'].astype(str)

       vizStory vizPageNumber  \
3050  Homestuck          3052   

                                         pageCommentary pageNotes  
3050  On the other hand, I guess Tavros did technica...      None  
     section  pageId title                                            content  \
5997   story  004952   ==>  |PESTERLOG|<br /><span style="color: #a15000">...   

       timestamp                        src      next previous theme flag  \
5997  1292098797  /storyfiles/hs2/03050.gif  [004953]   004951         []   

      ... vizPageId                           vizLink pageNumber  \
5997  ...    6/3052  https://homestuck.com/story/3052   6/004952   

     transcript_class transcript_style transcript_tag       transcript tags  \
5997              NaN              NaN            NaN  \nDOOF\n\nNAK\n   []   

                          pov_characters  \
5997  [Becquerel, Grandpa, Jade, Tavros]   

                                           pov_commands  
5997  [[Becquerel: Be controlled by 

In [527]:
# Merge the data into mspa_df_with_pov
mspa_df_with_pov_and_commentary = mspa_df_with_pov.merge(commentary_df, on=['vizStory', 'vizPageNumber'], how='left')

# Show some data with notes
mspa_df_with_pov_and_commentary[mspa_df_with_pov_and_commentary['pageNotes'].notnull()].sample(10)

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,...,pageNumber,transcript_class,transcript_style,transcript_tag,transcript,tags,pov_characters,pov_commands,pageCommentary,pageNotes
3633,story,002970,[S] Dave: Strife!,,1261643131,/storyfiles/hs2/01070/01070.swf,[002971],002969,,"[F, HQ, BOLIN, BOLINHQ]",...,6/002970,NaN,NaN,NaN,NaN,"[Part 1, Act 3]","[Bro, Dave, Lil' Cal]","[[Bro: Strife], [Dave: Duel Bro], [Lil' Cal: S...",0:15 - The crows do enjoy a good Strider beatd...,"Makin: This flash originally used the song ""Co..."
3499,story,002880,[S] Jade: Retrieve package.,,1260259150,/storyfiles/hs2/00980/00980.swf,[002881],002879,,"[F, HQ, BOLIN, BOLINHQ]",...,6/002880,NaN,NaN,NaN,NaN,"[Part 1, Act 3]","[Becquerel, Jade]","[[Becquerel: Play with Jade, Becquerel: Tuck J...","0:08 - Jade: Seriously, just walk over there a...","Makin: This flash originally used the song ""Mu..."
3328,story,002733,[S] MIDNIGHT CREW: ACT 1031,,1257152645,/storyfiles/hs2/00833/00833.swf,[002734],002732,,"[F, HQ, BOLIN, BOLINHQ]",...,6/002733,NaN,NaN,NaN,NaN,[],"[Biscuits, Cans, Clover, Clubs Deuce, Crowbar,...","[[Biscuits: Midnight Crew: Act 1031], [Cans: M...","The Midnight Crew have item/weapon duality, ju...",Makin: 1031 is just a reference to the flash b...
407,story,000310,Listen.,It's Ace Dick's office again. He is still on t...,1205900705,/advimgs/ps/ps092.gif,[000311],000309,retro,[],...,4/000310,NaN,NaN,NaN,NaN,[],[],[],One thing is clear: these detectives have an a...,"Drew Linky: If you count the flashing ""CUT SCE..."
3985,story,003252,[I] ==>,,1265745093,/storyfiles/hs2/01352_retcon.gif,[003253],003251,,[R4],...,6/003252,NaN,NaN,NaN,NaN,"[Part 1, Intermission, Oil, Retcon Oil]",[Spades Slick],"[[Spades Slick: Open safe, Spades Slick: Clock...",None,Makin: The donator that seeded the idea for th...
3702,story,003023,==>,"<a href=""http://www.mspaintadventures.com/swee...",1262649310,/storyfiles/hs2/01123_2.gif,[003024],003022,,[],...,6/003023,NaN,NaN,NaN,KLONK,"[Part 1, Act 3, Lined Mode, Carapacian, Dersit...","[Authority Regulator, Parcel Mistress, Warwear...",[[Authority Regulator: Open fire with missile ...,"Subtle trivia: Did you know the ""LoB"" sound ef...",Drew: I don't know about pressing it with your...
4605,story,003743,Dave: Combine fetus in a jar and self portrait...,That would apparently make DAVE'S BRAIN IN A J...,1273812843,/storyfiles/hs2/01843_2.gif,[003744],003742,,[],...,6/003743,NaN,NaN,NaN,NaN,"[Part 1, Act 4, Dave's Brain in a jar, Holopad...",[Dave],[[Dave: Wake up and jam.]],I don't remember if we ever found out what typ...,Drew: After all these years the mystery has be...
3314,story,002722,[S] Jade: Play a hauntingly relaxing bassline.,,1256635621,/storyfiles/hs2/00822/00822.swf,[002723],002721,,"[F, BOLIN]",...,6/002722,NaN,NaN,NaN,NaN,"[Part 1, Act 3]",[Jade],"[[Jade: Be introduced, Jade: Talk to Dave]]",Here we confirm that Jade lives on that island...,"Makin: This flash originally used the song ""Gu..."
4435,story,003620,[S] Rose and Dave: Shut up and jam.,Dave shows you some of his sweet gear. Wow he ...,1271898902,/storyfiles/hs2/01720/01720.swf,[003621],003619,,"[F, BOLIN]",...,6/003620,NaN,NaN,NaN,NaN,[],"[Dave, Lil' Cal, Rose]","[[Dave: Jam with Rose, Dave: See Rose throw dr...",This is a montage of Dave and Rose dancing to ...,"Makin: This flash originally used the songs ""B..."
406,story,000310,Listen.,It's Ace Dick's office again. He is still on t...,1205900705,/advimgs/ps/cutscene.gif,[000311],000309,retro,[],...,4/000310,NaN,NaN,NaN,CUT SCENE,[],[],[],One thing is clear: these detectives have an a...,"Drew Linky: If you count the flashing ""CUT SCE..."


# Cleaning up

We plan on converting this to CSV and potentially JSONL, so right now have a lot of object columns that should be strings, a bunch of empty strings and NaNs that should be pd.NAs. 

In [528]:
# mspa_df_with_pov currently has these columns representing MSPA data:
# section (str, per page): the section of the MSPA website the page is in, always a string like "story"
# pageId (str, per page): the page number, usually a six digit padded string like "001901", sometimes a regular string like "pony"
# title (str, per page): the title of the page, usually the command given in the previous page, always a string like "John: Enter name."
# content (str, per page): the content of the page, usually the page minus the panels (includes dialog and HTML)
# timestamp (timestamp, per page): the UNIX timestamp of the page, always a number like 1580000000
# src (str, per panel): the partial image file name, when prefixed with mspaintadventures.com or homestuck.com, something like "/storyfiles/hs2/001901.gif"
# next (list of strings, per page): the pageId of the next pages, usually a list of one string like ["001902"]
# previous (string, per page): the pageId of the previous page, usually a string like "001900"
# theme (str, per page): the theme of the page, like "A6A6", always a string, usually empty
# flag (list of strings, per page): a list of flags for Homestuck Collection presentation, some flags like ["R1"] show that the page is part of a "retcon" where Hussie went back and edited panels, others like ["FIREFLY"] show the page has special javascript that shows fireflies flying around the page, a page usually has no flag
# vizStory (str, per page): essentially a readable version of the section, always a string like "Homestuck"
# vizPageNumber (str, per page): pages are successive in MSPA but per section in VIZ, for example Homestuck's first page is 1901 in MSPA but 1 in VIZ, always a non-padded string since string pages like "pony" still exist
# vizPageId (str, per page): we should remove this column, being a combination of section number and viz page that was only used in one dataset we wanted to merge with
# vizLink (str, per page): the link to the page in the VIZ website, always a full URI like "https://homestuck.com/story/5320"
# pageNumber (str, per page): like vizPageId but for MSPA format, also should be removed
# transcript class (str, per panel): the class of the transcript, like "dialogue" or "narration", always a string, usually a NaN which should be replaced with None
# transcript style (str, per panel): the style of the transcript, in CSS, like "font-weight: bold", always a string, usually a NaN which should be replaced with None
# transcript tag (str, per panel): the tag of the transcript, like "p" or "div", always a string, usually a NaN which should be replaced with None
# transcript (str, per panel): the transcript of the panel, the text that appears in the panel, always a string, usually a NaN which should be replaced with None
# tags (list of strings, per page): the tags of the page, like ["John Egbert", "Rose Lalonde", "Land of Wind and Shade"], always a list of strings, usually an empty list
# pov_characters (list of strings, per page): the characters that appear in the page according to the POV cam, always a list of strings, usually an empty list
# pov_commands (list of strings, per page): the commands that the characters have in the page according to the POV cam, always a list of lists of strings (per character), usually an empty list
# pageCommentary (string, per page): the commentary of the page, usually by Andrew Hussie, always a string, usually a NaN or None which should be replaced with None
# pageNotes (string, per page): the notes of the page, usually by volunteers, always a string, usually a NaN or None which should be replaced with None

# Let's start by removing the columns we don't need
mspa_df_cleaned = mspa_df_with_pov_and_commentary.drop(columns=['vizPageId', 'pageNumber'])

# Replace NaN with pd.NA in the transcript columns
na_to_pdna_columns = ['transcript_class', 'transcript_style', 'transcript_tag', 'transcript', 'pageCommentary', 'pageNotes']
mspa_df_cleaned[na_to_pdna_columns] = mspa_df_cleaned[na_to_pdna_columns].where(mspa_df_cleaned[na_to_pdna_columns].notnull(), pd.NA)

# Replace empty strings with pd.NA in all columns
mspa_df_cleaned = mspa_df_cleaned.replace('', pd.NA)
        
# Show some cleaned data
mspa_df_cleaned.sample(10)

,section,pageId,title,content,timestamp,src,next,previous,theme,flag,...,vizLink,transcript_class,transcript_style,transcript_tag,transcript,tags,pov_characters,pov_commands,pageCommentary,pageNotes
7361,story,006117,==>,"|PESTERLOG|<br /><span style=""color: #00d5f2"">...",1322301349,/storyfiles/hs2/04214.gif,[006118],006116,<NA>,[],...,https://homestuck.com/story/4217,<NA>,<NA>,<NA>,<NA>,[],"[Jake, Jane]","[[Jake: Talk with Jane, Jake: Be told about de...",<NA>,<NA>
2806,story,002297,"[S] GET UP JOHN, THIS IS NO TIME FOR SLUMBER.",<NA>,1248391472,/storyfiles/hs2/00397/00397.swf,[002298],002296,<NA>,[F],...,https://homestuck.com/story/397,<NA>,<NA>,<NA>,<NA>,[],"[John, Warweary Villein]","[[John: Strife!], [Warweary Villein: BOY., War...","In this Flash, I think some people didn't quit...",<NA>
8732,story,007420,Jake: Pester Jane.,<NA>,1354313986,/storyfiles/hs2/05517.gif,[007421],007419,<NA>,[],...,https://homestuck.com/story/5520,<NA>,<NA>,<NA>,Alert : Jake,[],"[Fefetasprite, Jake, Jane, Roxy]","[[Fefetasprite: Be at Jane's birthday], [Jake:...",<NA>,<NA>
521,story,000387,Throw.,The bust crumbles against the rigid surface of...,1207343804,/advimgs/ps/ps169.gif,[000388],000386,retro,[],...,https://homestuck.com/problem-sleuth/169,<NA>,<NA>,<NA>,<NA>,[],[],[],More on this generator. I had even intended to...,<NA>
2700,story,002199,Rose: Allocate knitting needles to strife spec...,You feel a lot more comfortable with this as a...,1245885952,/storyfiles/hs2/00299.gif,[002200],002198,<NA>,[],...,https://homestuck.com/story/299,<NA>,<NA>,<NA>,\nstrife specibus\n\nsylladex :: strife deck\n...,"[Act 2, Part 1, Grimoire, Strife Specibus, Kni...",[Rose],[[Rose: Head outside]],<NA>,<NA>
4859,story,003888,Recap 2.,|RECAP LOG|<br />Picking up from where we left...,1276312597,/storyfiles/hs2/01988.gif,[003889],003887,<NA>,[],...,https://homestuck.com/story/1988,<NA>,<NA>,<NA>,TAP TAP BANG BANG,"[Part 1, Act 4, Scribble Mode, Andrew Hussie, ...",[],[],This is a pretty good way to handle this recap...,<NA>
7642,story,006393,==>,"|PESTERLOG|<br /><span style=""color: #ff6ff2"">...",1327469362,/storyfiles/hs2/04490.gif,[006394],006392,<NA>,[],...,https://homestuck.com/story/4493,<NA>,<NA>,<NA>,<NA>,[],[Roxy],"[[Roxy: Talk with Lil' Hal, Roxy: Redo Jane co...",<NA>,<NA>
7659,story,006407,Roxy: Break some glass.,<NA>,1327562429,/storyfiles/hs2/04504_2.gif,[006408],006406,<NA>,[],...,https://homestuck.com/story/4507,<NA>,<NA>,<NA>,<NA>,[],[Roxy],"[[Roxy: Talk with Lil' Hal, Roxy: Redo Jane co...",<NA>,<NA>
4949,story,003992,======>,"But Sollux, even with his vision twofold, does...",1277436573,/storyfiles/hs2/02092.gif,[003993],003991,<NA>,[],...,https://homestuck.com/story/2092,<NA>,<NA>,<NA>,<NA>,[],[Sollux],[[Sollux: Delete viruses]],"Is this the first use of the ""already here"" re...",<NA>
11247,story,009778,[A6A6I5] ====>,<NA>,1437347781,/storyfiles/hs2/07874.gif,[009779],009777,<NA>,[],...,https://homestuck.com/story/7878,<NA>,<NA>,<NA>,snikt!,[],"[Davepetasprite^2, Jake, Jasprosesprite^2]",[[Davepetasprite^2: claw claw meowtherfuckers!...,<NA>,<NA>


In [529]:
# Fix the inconsistent casing (some columns are snake_case, others camelCase)
# Annoyingly, there doesn't seem to be a standard for this for ML datasets, so we'll go with Python's snake_case
# This means altering pageId, vizStory, vizPageNumber, vizLink, pageCommentary, pageNotes

mspa_df_cleaned.rename(columns={
    'pageId': 'page_id',
    'vizStory': 'viz_story',
    'vizPageNumber': 'viz_page_number',
    'vizLink': 'viz_link',
    'pageCommentary': 'page_commentary',
    'pageNotes': 'page_notes'
}, inplace=True)

mspa_df_cleaned.sample(5)

,section,page_id,title,content,timestamp,src,next,previous,theme,flag,...,viz_link,transcript_class,transcript_style,transcript_tag,transcript,tags,pov_characters,pov_commands,page_commentary,page_notes
11320,story,009854,[A6A6I5] ====>,<NA>,1438030883,/storyfiles/hs2/07947.gif,[009855],009853,<NA>,[],...,https://homestuck.com/story/7954,<NA>,<NA>,<NA>,<NA>,[],[Terezi],"[[Terezi: Monologue to Vriska, Terezi: Do mind...",<NA>,<NA>
9284,story,007878,[A6I5] ==>,"|DIALOGLOG|<br /><span style=""color: #77003c"">...",1362937263,/storyfiles/hs2/05975.gif,[007879],007877,<NA>,[],...,https://homestuck.com/story/5978,<NA>,<NA>,<NA>,\nghost!Feferi: 38D\nghost!Nepeta: :33\n,[],"[Aradia, Aranea, Feferi, John, Meenah, Nepeta,...","[[Aradia: Be in Vriska's crew], [Aranea: Join ...",<NA>,<NA>
5447,story,004429,Terezi: Inquire.,<NA>,1282785681,/storyfiles/hs2/02528.gif,[004430],004428,<NA>,[],...,https://homestuck.com/story/2529,<NA>,<NA>,<NA>,TEREZI: T1M3 FOR WH4T?,[],"[Snowman, Terezi]","[[Snowman: Be exile for Vriska, Terezi], [Tere...",<NA>,<NA>
2417,story,001963,John: Check message.,"|PESTERLOG|<br /><span style=""color: #b536da"">...",1240192555,/storyfiles/hs2/00063.gif,[001964],001962,<NA>,[],...,https://homestuck.com/story/63,<NA>,<NA>,<NA>,\n:: tentacleTherapist ::\nPESTERLOG : ...,"[Part 1, Act 1, Screen, Desktop, Pesterchum, T...","[John, Rose]","[[John: Be introduced, John: Mess around], [Ro...",When Rose correctly guessed John was wearing a...,<NA>
6818,story,005681,[o] Tock.,<NA>,1305788954,/storyfiles/hs2/03778.gif,[005682],005680,scratch,[],...,https://homestuck.com/story/3781,<NA>,<NA>,<NA>,<NA>,[],"[Doc Scratch, Jack Noir, Rose]",[[Doc Scratch: The dÎsc is tø€ badly da™aged. ...,<NA>,<NA>


In [530]:
# Make column types explicit as verification and so they don't resolve to float when we save them, and timestamp stays a timestamp

mspa_df_cleaned['timestamp'] = pd.to_datetime(mspa_df_cleaned['timestamp'], unit='s')

# Convert all columns to string except for the lists and timestamp

for column in mspa_df_cleaned.columns:
    if column not in ['timestamp', 'next', 'tags', 'pov_characters', 'pov_commands', 'flag']:
        mspa_df_cleaned[column] = mspa_df_cleaned[column].astype('string')
        
mspa_df_cleaned.sample(5)

C:\Users\PC\AppData\Local\Temp\ipykernel_2528\1107193909.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  mspa_df_cleaned['timestamp'] = pd.to_datetime(mspa_df_cleaned['timestamp'], unit='s')


,section,page_id,title,content,timestamp,src,next,previous,theme,flag,...,viz_link,transcript_class,transcript_style,transcript_tag,transcript,tags,pov_characters,pov_commands,page_commentary,page_notes
3470,story,002853,Enter name.,Spades Slick?<br /><br />Got a nice ring to it...,2009-12-03 20:01:36,/storyfiles/hs2/00953_1.gif,[002854],002852,<NA>,[],...,https://homestuck.com/story/953,<NA>,<NA>,<NA>,> SPADES SLICK,"[Part 1, Act 3, Jack Noir, Fenestrated Wall, D...",[Jack Noir],[[Jack Noir: Dad breaking out of prison]],<NA>,<NA>
4919,story,003966,A little later...,Somewhere else entirely...,2010-06-21 17:52:46,/storyfiles/hs2/02066.gif,[003967],003965,<NA>,[],...,https://homestuck.com/story/2066,<NA>,<NA>,<NA>,Big 24 oz,[],[Eridan],[[Eridan: Teaser introduction]],There's the Faygo that Gamzee launched into th...,<NA>
11522,ryanquest,000015,Ryan: Consume meal.,You chow down on ONE (1) HOT TURKEY PLATTER WI...,NaT,/ryanquest/15.gif,[],000014,<NA>,[],...,https://homestuck.com/ryanquest/15,<NA>,<NA>,<NA>,<NA>,[],[],[],<NA>,<NA>
3681,story,003010,==>,ORDER IN THE COURT. YOU WILL HAVE ORDER IN THI...,2010-01-03 22:59:48,/storyfiles/hs2/01110.gif,[003011],003009,<NA>,[],...,https://homestuck.com/story/1110,<NA>,<NA>,<NA>,Judge Jury,"[Part 1, Act 3, Carapacian, Dersite, Exile, AR...",[Authority Regulator],[[Authority Regulator: Seek more powerful weap...,<NA>,<NA>
8204,story,006920,"Jake: Ok, try out infinitesimalizer.",You deploy the massive thing that's been clogg...,2012-06-04 02:49:39,/storyfiles/hs2/05017.gif,[006921],006919,<NA>,[],...,https://homestuck.com/story/5020,<NA>,<NA>,<NA>,<NA>,[],[Jake],"[[Jake: Wake up., Jake: Read note left by Call...",<NA>,<NA>


In [531]:
# Let's reorder the columns to make it easier to read, the first columns should let a user know the story and the title of the page at least

mspa_df_cleaned = mspa_df_cleaned[['viz_story', 'title', 'content', 'page_id', 'timestamp', 'src', 'next', 'previous', 'viz_page_number', 'viz_link', 'tags', 'pov_characters', 'pov_commands', 'transcript_class', 'transcript_style', 'transcript_tag', 'transcript', 'page_commentary', 'page_notes', 'theme', 'flag']]

mspa_df_cleaned.sample(5)

,viz_story,title,content,page_id,timestamp,src,next,previous,viz_page_number,viz_link,...,pov_characters,pov_commands,transcript_class,transcript_style,transcript_tag,transcript,page_commentary,page_notes,theme,flag
7882,Homestuck,[A6I2] ==>,"|DIALOGLOG|<br /><span style=""color: #e00707"">...",006612,2012-03-18 01:59:07,/storyfiles/hs2/04709.gif,[006613],006611,4712,https://homestuck.com/story/4712,...,"[Dave, Kanaya, Karkat, Rose]","[[Dave: Talk to Rose, Kanaya, Dave: Learn abou...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[]
7022,Homestuck,[o] ==>,<NA>,005847,2011-07-14 21:00:22,/storyfiles/hs2/03944.gif,[005848],005846,3947,https://homestuck.com/story/3947,...,"[Davesprite, Jadesprite]",[[Davesprite: Have Jadesprite teleport sword t...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,scratch,[]
4536,Homestuck,Rose: Combine hubtop and hair band.,"You make the HUBTOPBAND, a convenient hands-fr...",003707,2010-05-06 23:47:22,/storyfiles/hs2/01807_1.gif,[003708],003706,1807,https://homestuck.com/story/1807,...,[Rose],"[[Rose: Alchemize, Rose: Create Thorns of Oglo...",<NA>,<NA>,<NA>,HUBTOPBAND □ 500 ▽ 200,"Wait, we're back in action. A hands-free commu...",<NA>,<NA>,[]
3704,Homestuck,==>,<NA>,003025,2010-01-05 02:04:10,/storyfiles/hs2/01125_2.gif,[003026],003024,1125,https://homestuck.com/story/1125,...,"[Authority Regulator, Parcel Mistress, Warwear...",[[Authority Regulator: Open fire with missile ...,<NA>,<NA>,<NA>,SHOOOOO,I think AR may be getting an unfair share of t...,<NA>,<NA>,[]
973,Problem Sleuth,Next.,<NA>,000817,2008-08-14 02:53:45,/advimgs/ps/ps605.gif,[000818],000816,599,https://homestuck.com/problem-sleuth/599,...,[],[],<NA>,<NA>,<NA>,GAMESPOT GameFAQs Email Password Sign Up | ...,"This is complete titularity. ChaosDemon, Sanc...",<NA>,retro,[]


In [532]:
# And let's make a dict with column descriptions for later when we release the dataset

dataset_column_descriptions = {
    'viz_story': "The story the page is part of, always 'Homestuck'.",
    'title': "The title of the page, usually the command given in the previous page.",
    'content': "The content of the page, usually the page minus the panels (includes dialog and HTML).",
    'page_id': "The page number, usually a six digit padded string like '001901', sometimes a regular string like 'pony'.",
    'timestamp': "The UNIX timestamp of the page.",
    'src': "The partial image file name, when prefixed with mspaintadventures.com or homestuck.com, something like '/storyfiles/hs2/001901.gif'.",
    'next': "The pageId of the next pages, usually a list of one string like ['001902'].",
    'previous': "The pageId of the previous page, usually a string like '001900'.",
    'viz_page_number': "Pages are successive in MSPA but per section in VIZ, for example Homestuck's first page is 1901 in MSPA but 1 in VIZ.",
    'viz_link': "The link to the page in the VIZ website.",
    'tags': "The tags of the page, like ['John Egbert', 'Rose Lalonde', 'Land of Wind and Shade'].",
    'pov_characters': "The characters that appear in the page according to the POV cam.",
    'pov_commands': "The commands that the characters have in the page according to the POV cam.",
    'transcript_class': "The class of the transcript, like 'dialogue' or 'narration'.",
    'transcript_style': "The style of the transcript, in CSS.",
    'transcript_tag': "The tag of the transcript, like 'p' or 'div'.",
    'transcript': "The transcript of the panel, the text that appears in the panel.",
    'page_commentary': "The commentary of the page, usually by Andrew Hussie.",
    'page_notes': "The notes of the page, usually by volunteers.",
    'theme': "The theme of the page, like 'A6A6'.",
    'flag': "A list of flags for Homestuck Collection presentation."
}

In [533]:
# Before we continue we need to confirm we can fetch the image or asset of each panel
# Pick an asset with a src, like "/storyfiles/hs2/001901.gif"

# We'll use the requests library to fetch the image
import requests

# We just need to display the image in a Jupyter notebook so we'll use IPython's display
from IPython.display import Image

# We'll also use BytesIO to convert the image to bytes
from io import BytesIO

# We'll use a random homestuck panel with enough metadata, like the one in viz page 2519
panel_src = mspa_df_cleaned[mspa_df_cleaned['viz_page_number'] == '2519']['src'].values[0]
panel_url = f"https://www.homestuck.com/images{panel_src}"

# Test the image URL
print(f"Fetching image from {panel_url}")
requests.get(panel_url)


Fetching image from https://www.homestuck.com/images/storyfiles/hs2/02518.gif


<Response [200]>

In [534]:
Image(url=panel_url)

In [535]:
# Right now we haven't even saved the data, so let's do that first, as csv

mspa_df_cleaned.to_csv('mspa_dataset.csv', index=False)

# Now we can load the data back (CSV does have the limitation of not representing pd.NA correctly though, we'll just get NaNs)
mspa_df_csv = pd.read_csv('mspa_dataset.csv')
mspa_df_csv.sample(5)

,viz_story,title,content,page_id,timestamp,src,next,previous,viz_page_number,viz_link,...,pov_characters,pov_commands,transcript_class,transcript_style,transcript_tag,transcript,page_commentary,page_notes,theme,flag
7985,Homestuck,[A6I2] ==>,NaN,006714,2012-04-02 02:56:44,/storyfiles/hs2/04811.gif,['006715'],6713.0,4814,https://homestuck.com/story/4814,...,[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
1964,Problem Sleuth,Next.,"The BARREL bounces off PI's head, sailing over...",001640,2009-01-26 03:55:00,/advimgs/ps/ps1427_2.gif,['001641'],1639.0,1422,https://homestuck.com/problem-sleuth/1422,...,[],[],NaN,NaN,NaN,NaN,NaN,NaN,retro,[]
9012,Homestuck,GO BACK TO ACT 6 ACT 5 ACT 1.,You return to the safety of ACT 6 ACT 5 ACT 1....,007678,2013-01-26 04:17:43,/storyfiles/hs2/05775.gif,['007679'],7677.0,5778,https://homestuck.com/story/5778,...,"['Equiusprite', 'Caliborn', 'Dirk', 'Erisolspr...","[['Equiusprite: Witness arival of tricksters',...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
5545,Homestuck,======>,NaN,004523,2010-09-18 17:01:30,/storyfiles/hs2/02622.gif,['004524'],4522.0,2623,https://homestuck.com/story/2623,...,"['Aradia', 'Sollux']","[['Aradia: Arrive on meteor'], ['Sollux: Arriv...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
7208,Homestuck,= >= <<,NaN,006005,2011-10-25 04:51:08,/storyfiles/hs2/04102_2.gif,['006006'],6004.0,4105,https://homestuck.com/story/4105,...,"['Courtyard Droll', 'Jack Noir', 'Jade']",[['Courtyard Droll: Succeed in actual mission'...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]


# Tagging panels

We have the panel images, we have the ReadMSPA transcripts of them, we have the point of view from the POV cam, and we have the title and text that accompanies the panels. We have some partial tagging information from the Homestuck Search engine. With that and a vision model, we might be able to successfully extract non-hallucinated information.
Basically we need to predict the tag-related columns, since we only have tagging data for the first four acts.

In [536]:
# First we need to get only the image we need. We can assemble the image URLs from the src column, by prefixing "https://homestuck.com/images" to it
# That, plus the rest of informational columns, will be useful information to train the model with
# We want to predict the "tags" column given the rest, so we take the Act 1-4 data (basically everything that has a non-empty tags column):

four_acts_hs_df = mspa_df_cleaned[mspa_df_cleaned['tags'].apply(len) > 0]
four_acts_hs_df.sample(5)

,viz_story,title,content,page_id,timestamp,src,next,previous,viz_page_number,viz_link,...,pov_characters,pov_commands,transcript_class,transcript_style,transcript_tag,transcript,page_commentary,page_notes,theme,flag
3026,Homestuck,Dave: Press EJECT button.,Oh hell no. Not after all that trouble you wen...,002484,2009-09-04 22:41:54,/storyfiles/hs2/00584.gif,[002485],002483,584,https://homestuck.com/story/584,...,[Dave],[[Dave: Puppets]],<NA>,<NA>,<NA>,[EJECT] EJECT ALL ITEMS FROM SYLLADEX? [Y] ...,<NA>,<NA>,<NA>,[]
2706,Homestuck,==>,"And NRUB'YIGLITH, SHAMEBEAST KING OF GROTESQUE...",002203,2009-06-25 22:31:41,/storyfiles/hs2/00303.gif,[002204],002202,303,https://homestuck.com/story/303,...,[Rose],[[Rose: Head outside]],<NA>,<NA>,<NA>,NRUB'yiglith,<NA>,<NA>,<NA>,[]
3648,Homestuck,Dave: Go get a god damn new sword.,"Perhaps you will. But if you do, it looks like...",002983,2009-12-31 07:27:43,/storyfiles/hs2/01083.gif,[002984],002982,1083,https://homestuck.com/story/1083,...,"[Dave, Lil' Cal]","[[Dave: Mourn loss of Lil' Cal], [Lil' Cal: Be...",<NA>,<NA>,<NA>,strife specibus sylladex :: strife deck 1/2...,<NA>,<NA>,<NA>,[]
2569,Homestuck,John: Captchalogue telescope.,You snatch the TELESCOPE from its TRIPOD. Who ...,002090,2009-05-22 01:17:33,/storyfiles/hs2/00190_1.gif,[002091],002089,190,https://homestuck.com/story/190,...,"[John, Rose]","[[John: Get trapped in room], [Rose: Connect t...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[]
2841,Homestuck,==>!,"|SPRITELOG|<br /><span style=""color: #0715cd"">...",002326,2009-08-03 02:34:55,/storyfiles/hs2/00426.gif,[002327],002325,426,https://homestuck.com/story/426,...,"[John, Nannasprite, Rose, Warweary Villein]","[[John: Notice door is fixed, John: Be pranked...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[]


In [537]:
# For the dataset we'll need to make a new dataframe with only the relevant data
# We don't need viz_story, timestamp, next, previous, viz_page_number, viz_link
# We'll keep the rest, and remove the tag later when it's time

tagging_dataset = four_acts_hs_df.drop(columns=['viz_story', 'timestamp', 'next', 'previous', 'viz_page_number', 'viz_link'])

tagging_dataset.sample(5)

,title,content,page_id,src,tags,pov_characters,pov_commands,transcript_class,transcript_style,transcript_tag,transcript,page_commentary,page_notes,theme,flag
2364,John: Use hammer/nails on poster.,You use the HAMMER and NAILS card IN CONJUNCTI...,001918,/storyfiles/hs2/00018.gif,"[Part 1, Act 1, Earth, John's House, John's Ro...",[John],"[[John: Be introduced, John: Mess around]]",<NA>,<NA>,<NA>,<NA>,Carefully educating you on needlessly complica...,<NA>,<NA>,[]
4124,AR?: Write the owner of this vehicle a ticket.,Hello. What have we here?,003358,/storyfiles/hs2/01458_1.gif,"[Part 1, Act 4, Car, Boondollar, Package]",[Authority Regulator],[[Authority Regulator: Ticket car]],<NA>,<NA>,<NA>,"PARKING CITATION B$10,000",There's nothing at all that isn't funny about ...,<NA>,<NA>,[]
4103,==>,Once you're done you'll captchalogue the bottl...,003341,/storyfiles/hs2/01441_2.gif,"[Part 1, Act 4, John Egbert, Apple Juice, Capt...",[Dave],[[Dave: Talk to dream Jade]],<NA>,<NA>,<NA>,→ SOME CODE → PESTERCHUM hey dude here is a ...,<NA>,<NA>,<NA>,[R4]
3903,[I] PSS: Be Future Spades Slick.,Being your future self is a lot more construct...,003189,/storyfiles/hs2/01289.gif,"[Part 1, Intermission, Felt Manor, Clock, Bloo...","[Sawbuck, Spades Slick]","[[Sawbuck: Be interrupted by Snowman, Sawbuck:...",<NA>,<NA>,<NA>,<NA>,The first sentence here would be quite a wise ...,<NA>,<NA>,[]
3195,WV: Mourn the loss of citizen tab.,Your caffeinated jittering must have agitated ...,002622,/storyfiles/hs2/00722_1.gif,"[Part 1, Act 2, Skyship Base, Command Station,...",[Warweary Villein],"[[Warweary Villein: WV: Retri..., Ascend.]]",<NA>,<NA>,<NA>,<NA>,WV: Now speculate on why you bothered to specu...,<NA>,<NA>,[]


In [538]:
"""
Now let's turn it into OpenAI's JSONL format, an example of which is here:

```jsonl
{
  "messages": [
    { "role": "system", "content": "You are an assistant that identifies uncommon cheeses." },
    { "role": "user", "content": "What is this cheese?" },
    { "role": "user", "content": [
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/3/36/Danbo_Cheese.jpg"
          }
        }
      ]
    },
    { "role": "assistant", "content": "Danbo" }
  ]
}
```

Each row of the dataset will turn into something here. 
The system prompt will contain an example of the input data, so let's deal with that later.
The first user content will be the metadata minus the image and the tags.
The second user content will be the image URL, which we can once again get from https://homestuck.com/images plus the src column. No need to base64 encode it.
The assistant content will be the tags column.
"""

system_prompt = """You are a professional image tagger.
Your current project is tagging the panels of the webcomic Homestuck. Thankfully, for each panel we have the image AND useful page and panel metadata.
You will be given input that contains transcripts of the text onscreen, the content under the panels with dialogue and narration, and some more metadata.
The image alongside will be the panel itself. Here's an example of the metadata input:
```
{"title": "Jade: Keep going.", "content": "Oh yeah. How could you forget about his stash of DECREPIT MUMMIES.<br /><br />God you hate these things.", "page_id": "002753", "src": "/storyfiles/hs2/00853_1.gif", "pov_characters": ["Jade"], "pov_commands": [["Jade: Proceed through house", "Jade: Be trolled by Karkat"]], "transcript_class": null, "transcript_style": null, "transcript_tag": null, "transcript": null, "page_commentary": null, "page_notes": null, "theme": null, "flag": []}
```
And an example of your tagged output, which must be a valid JSON list:
```
["Jade Harley", "Jade's House", "Blue Sun Shirt", "Stairs", "Dead", "Light Symbol"]
```
"""

def create_openai_jsonl_entry(row: pd.Series) -> dict:
    """
    Creates an OpenAI JSONL entry from a row of the tagging dataset.

    Args:
        row (pd.Series): A row of the tagging dataset.

    Returns:
        dict: An OpenAI JSONL entry.
    """
    # We must remove the organizational tags like "Act 1" and "Part 1" from the tags, because
    # the model cannot predict them
    usable_tags = [tag for tag in row['tags'] if "Act " not in tag and "Part " not in tag]
    # Tags will also be stringified so the model can predict them
    usable_tags = json.dumps(usable_tags)
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": json.dumps(row.drop('tags').to_dict())},
         {"role": "user", "content": [
            {
                "type": "image_url",
                "image_url": {
                    "url": f"https://homestuck.com/images{row['src']}"
                }
            }
        ]},
        {"role": "assistant", "content": usable_tags}
    ]
    return {"messages": messages}

# Test it with one row
create_openai_jsonl_entry(tagging_dataset.iloc[1000])

{'messages': [{'role': 'system',
   'content': 'You are a professional image tagger.\nYour current project is tagging the panels of the webcomic Homestuck. Thankfully, for each panel we have the image AND useful page and panel metadata.\nYou will be given input that contains transcripts of the text onscreen, the content under the panels with dialogue and narration, and some more metadata.\nThe image alongside will be the panel itself. Here\'s an example of the metadata input:\n```\n{"title": "Jade: Keep going.", "content": "Oh yeah. How could you forget about his stash of DECREPIT MUMMIES.<br /><br />God you hate these things.", "page_id": "002753", "src": "/storyfiles/hs2/00853_1.gif", "pov_characters": ["Jade"], "pov_commands": [["Jade: Proceed through house", "Jade: Be trolled by Karkat"]], "transcript_class": null, "transcript_style": null, "transcript_tag": null, "transcript": null, "page_commentary": null, "page_notes": null, "theme": null, "flag": []}\n```\nAnd an example of you

In [539]:
# OpenAI's platform lets you upload two jsonl files, one with the training data and another with the validation data
# We'll split the dataset into 80% training and 20% validation, but we need to shuffle it first

shuffled_tagging_dataset = tagging_dataset.sample(frac=1, random_state=42)
training_size = int(len(shuffled_tagging_dataset) * 0.8)
training_data = shuffled_tagging_dataset.iloc[:training_size]
validation_data = shuffled_tagging_dataset.iloc[training_size:]

# Remove the training and validation files if they don't already exist
# Remember we're storing them in output, as "hs_tagging_tranining_data.jsonl" and "hs_tagging_validation_data.jsonl"

if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)
    
# Remove the files if they exist
training_file = os.path.join(OUTPUT_FOLDER, "hs_tagging_training_data.jsonl")
validation_file = os.path.join(OUTPUT_FOLDER, "hs_tagging_validation_data.jsonl")

if os.path.exists(training_file):
    os.remove(training_file)
if os.path.exists(validation_file):
    os.remove(validation_file)

# Create the training and validation jsonl files
with open(training_file, 'w') as training_jsonl, open(validation_file, 'w') as validation_jsonl:
    for _, row in training_data.iterrows():
        training_jsonl.write(json.dumps(create_openai_jsonl_entry(row)) + '\n')
    print(f"Training data written to {training_file}")
    for _, row in validation_data.iterrows():
        validation_jsonl.write(json.dumps(create_openai_jsonl_entry(row)) + '\n')
    print(f"Validation data written to {validation_file}")
        
# Let's check the first few lines of the training file
with open(training_file, 'r') as file:
    for _ in range(5):
        print(file.readline())
        
# Let's check the first few lines of the validation file
with open(validation_file, 'r') as file:
    for _ in range(5):
        print(file.readline())

Training data written to ./output\hs_tagging_training_data.jsonl
Validation data written to ./output\hs_tagging_validation_data.jsonl
{"messages": [{"role": "system", "content": "You are a professional image tagger.\nYour current project is tagging the panels of the webcomic Homestuck. Thankfully, for each panel we have the image AND useful page and panel metadata.\nYou will be given input that contains transcripts of the text onscreen, the content under the panels with dialogue and narration, and some more metadata.\nThe image alongside will be the panel itself. Here's an example of the metadata input:\n```\n{\"title\": \"Jade: Keep going.\", \"content\": \"Oh yeah. How could you forget about his stash of DECREPIT MUMMIES.<br /><br />God you hate these things.\", \"page_id\": \"002753\", \"src\": \"/storyfiles/hs2/00853_1.gif\", \"pov_characters\": [\"Jade\"], \"pov_commands\": [[\"Jade: Proceed through house\", \"Jade: Be trolled by Karkat\"]], \"transcript_class\": null, \"transcrip

In [540]:
# Queries to send to the annotation model
batch_queries = []
system_prompt =  """You are a professional image annotator.
Your current project is annotating the panels of the webcomic Homestuck. You need to write a textual description as well as a list of location and character tags.
Your input will some metadata like this:
{
    "src": "005624.gif",
    "title": "Jane: Reply",
    "pov_characters": ["Jane Crocker", "Caliborn"],
    "transcript": ["...", "HELP"]
}
This is an example output:
{
    "": ["Jane Crocker"],
    "locations": ["Land of Crypts and Helium", "Jane's House"],  # If you don't know the location, just leave it out
    "description": "Jane Crocker stands in the middle of her room, next to her bed. Jane is wearing a gray shirt with a blue monster logo on it, as well as a blue skirt. The room contains posters of movies. Outside the window we can see the Land of Crypts and Helium, a gray planet with multicolored flowers. There's a text bubble with '...' pointing to her head.",
}
You should write verbose descriptions that will be useful for people who can't see the image, as well as for training image models.
No talk; just go.
"""

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")
    

def annotate_panel(panel_data: dict, image_path: str) -> dict:
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": [
            {
                "type": "text",
                "text": json.dumps(panel_data)
            },
            {
                "type": "image_url", 
                "image_url": {
                    "url": f"data:image/png;base64,{encode_image(image_path)}"
                }
            }
        ]}
    ]

    response = openai_client.chat.completions.create(
        model=MODEL_ID,
        messages=messages,
        temperature=0.0,
    )
    return response.choices[0].message.content

def get_panel_data(panel_id: str) -> dict:
    # We get metadata from a variety of sources
    # src: the image file name
    # title, page_content: from the json archive
    # pov_characters: from the POV extension
    # transcript: from the ReadMSPA transcripts
    pass
    

# We load a test panel, from HS_PANELS_FOLDER, 01691.gif which should depict a sleeping Rose and an awake John
test_panel = os.path.join(HS_PANELS_FOLDER, "01691.gif")
test_panel_data = {
    "src": "01691.gif",
    "title": "John: Get up.",
    "page_content": """Despite the pandemonium of your entrance, Rose is still sound asleep. She must be really tuckered out!
<br>
<br>It looks like this little guy is awake and ready for action though. He is adorable. You decide to name him Dr. Meowgon Spengler.""",
    "pov_characters": ["John Egbert", "Rose Lalonde"],
    "transcript": ["Z"],
}

# We'll use the OpenAI API to annotate the panel
# This is a test, so we'll just print the output
# print(annotate_panel(test_panel_data, test_panel))

# Textual datasets

We output datasets for Homestuck, MS Paint Adventures as a whole, and all of Andrew Hussie's works.

The format is .jsonl